In [1]:
import re
from greek_normalisation.utils import (grave_to_acute,strip_last_accent_if_two)

lexicalentries = {}
verbarguments = {}
lexfile = "/Users/gcrane/github/Thucydides-new-working-materials/GreekMorphApril18.csv"
with open(lexfile) as f:
    for line in f:

        #print(line)
        #line = line.strip('"')
        fields = line.split(",")
        fields[1] = fields[1].strip('"')
        fields[2] = fields[2].strip('\n')
        fields[2] = fields[2].strip('"')
        lexicalentries[fields[1]] = fields[2]

f.close()

anabasistb = '/Users/gcrane/github/glaux-trees/public/xml/0032-006.xml'
republictb = '/Users/gcrane/github/glaux-trees/public/xml/0059-030.xml'
iliadtb = '/Users/gcrane/github/gAGDT/data/xml/tlg0012.tlg001.perseus-grc1.tb.xml'

curtb = anabasistb
#curtb = iliadtb

greeksentids = {}
greeksentforms = {}
greeksentlems = {}
greeksentlens = {}
greeksentpostags = {}
greeksentheads = {}
greeksentrels = {}
engsents = {}
engsentcits = {}
engsentlens = {}

greekforms = {}
greeklems = {}
hashead = {}
hasdeps = {}


neutvocsg = 's---nv'
neutnomsg = 's---nn'
neutgensg = 's---ng'
neutdatsg = 's---nd'
neutaccsg = 's---na'

neutvocpl = 'p---nv'
neutnompl = 'p---nn'
neutgenpl = 'p---ng'
neutdatpl = 'p---nd'
neutaccpl = 'p---na'

femvocsg = 's---fv'
femnomsg = 's---fn'
femgensg = 's---fg'
femdatsg = 's---fd'
femaccsg = 's---fa'

femvocpl = 'p---fv'
femnompl = 'p---fn'
femgenpl = 'p---fg'
femdatpl = 'p---fd'
femaccpl = 'p---fa'

mascvocsg = 's---mv'
mascnomsg = 's---mn'
mascgensg = 's---mg'
mascdatsg = 's---md'
mascaccsg = 's---ma'

mascvocpl = 'p---mv'
mascnompl = 'p---mn'
mascgenpl = 'p---mg'
mascdatpl = 'p---md'
mascaccpl = 'p---ma'

commvocsg = 's---cv'
commnomsg = 's---cn'
commgensg = 's---cg'
commdatsg = 's---cd'
commaccsg = 's---ca'

commvocpl = 'p---cv'
commnompl = 'p---cn'
commgenpl = 'p---cg'
commdatpl = 'p---cd'
commaccpl = 'p---ca'


mascparad = [mascnomsg,mascgensg,mascdatsg,mascaccsg,mascnompl,mascgenpl,mascdatpl,mascaccpl]

defart_forms = {
                mascvocsg : 'ὦ', mascnomsg : 'ὁ',  mascgensg : 'τοῦ', mascdatsg :'τῷ',  mascaccsg :'τὸν', 
                mascvocpl : 'ὦ', mascnompl: 'οἱ', mascgenpl : 'τῶν',mascdatpl : 'τοῖς', mascaccpl : 'τοὺς',
                femvocsg : 'ὦ', femnomsg : 'ἡ',  femgensg : 'τῆς', femdatsg :'τῇ',  femaccsg :'τὴν', 
                femvocpl : 'ὦ', femnompl: 'αἱ', femgenpl : 'τῶν',femdatpl : 'ταῖς', femaccpl : 'τὰς',
                neutvocsg : 'ὦ', neutnomsg : 'τὸ',  neutgensg : 'τοῦ', neutdatsg :'τῷ',  neutaccsg :'τὸ', 
                neutvocpl : 'ὦ', neutnompl: 'τὰ', neutgenpl : 'τῶν',neutdatpl : 'τοῖς', neutaccpl : 'τὰ',
                commvocsg : 'ὦ', commnomsg : 'ὁ/ἡ/τὸ',  commgensg : 'τοῦ/τῆς', commdatsg :'τῷ/τῇ',  commaccsg :'τὸν/τὴν/τὸ', 
                commvocpl : 'ὦ', commnompl: 'οἱ/αἱ/τὰ', commgenpl : 'τῶν',commdatpl : 'τοῖς/ταῖς', commaccpl : 'τὰ/τὰς/τοὺς'
               }

lemma_pos_forms = {}



def getengsents(fname):
    f = open(fname)
    for l in f:
        l = re.sub('\n','',l)
        l = re.sub('"','',l)
        args = l.split('\t')
        if(not len(args) == 3):
            print(len(args),l)
            continue
        engsents[args[1]] = args[2]
        engsentcits[args[1]] = args[0]
    
    f.close()
    
def additem(curdict,curitem):
    if(curitem in curdict):
        curdict[curitem] = curdict[curitem] + 1
    else:
        curdict[curitem] = 1
        

getengsents('/Users/gcrane/github/anabasis/sentfile.txt')  

depsdict = {}
formdict = {}
citdict = {}
sentdict = {}
def additem2list(curdict,hasdeps,newdep):
    if(not hasdeps in curdict):
        curdict[hasdeps] = []
    curdict[hasdeps].append(newdep)

def getgreeksents(fname):
    f = open(fname)
    cursent = ''
    i = 0
    for l in f:
        l = re.sub('\n','',l)
        m = re.search('<sentence id="([0-9]+)"',l)
        if(m):
            if(cursent):
                i = i + 1
                #if(i<100):
                #    print(l)
                #    print(' '.join(greeksentforms[cursent]))
            cursent = m[1]
            greeksentforms[cursent] = []
            greeksentlems[cursent] = []
            greeksentpostags[cursent] = []
            greeksentrels[cursent] = []
        m = re.search('subdoc="([^"]+)"',l)
        if(m):
            curcit = m[1]
        m = re.search('word id="([^"]+)" form="([^"]+)".+lemma="([^"]+)" postag="([^"]+)" head="([^"]+)" relation="([^"]+)"',l)
        if(m):
            curid = m[1]
            curform = m[2]
            curlemma = m[3]
            curpostag = m[4]
            curhead = m[5]
            currel = m[6]
            hashead[curid] = curhead
            if(curhead in hasdeps):
                hasdeps[curhead].append(curid)
            else:
                hasdeps[curhead] = [curid]
            
            if(not curlemma in lemma_pos_forms):
                lemma_pos_forms[curlemma] = {}
            additem(lemma_pos_forms[curlemma], curpostag + ':' + strip_last_accent_if_two(grave_to_acute(curform)))
            
            greeksentforms[cursent].append(curform)
            additem(greekforms,curform)
            greeksentlems[cursent].append(curlemma)
            additem(greeklems,curlemma)
            greeksentpostags[cursent].append(curpostag)
            greeksentrels[cursent].append(currel)
            if(not curhead == '0'):
                additem2list(depsdict,curhead,curid)
            formdict[curid] = l
            citdict[curid] = curcit
    f.close()
    print('len',len(lemma_pos_forms))
    for foo in greeksentlems:
        greeksentlens[foo] = len(greeksentlems[foo])

def get_relation(l):
    m = re.search('relation="([^"]+)"',l)
    if(m):
        return(m[1])
    else:
        return('')
    
def get_postag(l):
    m = re.search('postag="([^"]+)"',l)
    if(m):
        return(m[1])
    else:
        return('')
    
def get_lemma(l):
    m = re.search('lemma="([^"]+)"',l)
    if(m):
        return(m[1])
    else:
        return('')

def get_curid(l):
    m = re.search('id="([^"]+)"',l)
    if(m):
        return(m[1])
    else:
        return('')

def is_decl1(curlemma):
    if(re.search('(ος|ός)$',curlemma)):
        return(1)
    return(0)

def has_matchingarticle(headid,matchpos):
    if( not headid in depsdict):
        return
    for foo in depsdict[headid]:
        l = formdict[foo]
        if(re.search('lemma="ὁ"',l) and re.search('relation="ATR',l) and re.search(matchpos,l)):
            return(get_curid(l))
    return('')
 
def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

stoplist = ['ὁ','καί','οὐκ', 'δέ','εἰμί','ἄπειμι']

def searchvoc(searchlemlist,posfilts):
    #searchlem = searchlemlist[2]
    i = 0
    outf = open('currquest.tsv','w')
    #if(searchlem in greekforms and not searchlem in greeklems):
    #if(searchlem in greekforms and not intersection(searchlemlist,  greeklems)):
    if(0 and not intersection(searchlemlist,  greeklems)):
        print('sawform',searchlem)
        for foo in sorted(greeksentlens,key=greeksentlens.get):
            if(searchlem in greeksentforms[foo]):
                i = i + 1
                ind = greeksentforms[foo].index(searchlem)
                print(i,engsentcits[foo],len(greeksentlems[foo]),foo,greeksentforms[foo][ind],greeksentpostags[foo][ind],greeksentrels[foo][ind])
                print(' '.join(greeksentforms[foo]))
                print(engsents[foo])
                print(' '.join(greeksentlems[foo]))
                print()
    else:
        for foo in sorted(greeksentlens,key=greeksentlens.get):
            #if(searchlem in greeksentlems[foo]):
            interl = intersection(searchlemlist,  greeksentlems[foo])
            if(interl and posfilts):
                workinterl = []
                if(len(posfilts)>1):
                    possearch = '('+'|'.join(posfilts) + ')'
                else:
                    possearch = posfilts[0]
                for cursearchlem in interl:
                    ind = greeksentlems[foo].index(cursearchlem)
                    if(re.search(possearch,greeksentpostags[foo][ind])):
                        workinterl.append(cursearchlem)
                interl = workinterl
            if(interl):
                newlist = []
                for goo in greeksentforms[foo]:
                    newlist.append(goo)
                #newlist = greeksentforms[foo]
                i = i + 1
                print(i,interl)
                answers = []
                sentrels = []
                for cursearchlem in interl:
                    if(cursearchlem in stoplist):
                        continue

                    ind = greeksentlems[foo].index(cursearchlem)
                    #newlist = greeksentforms[foo]
                    print('new',newlist,newlist[ind])
                    answers.append(newlist[ind]) 
                    sentrels.append(greeksentrels[foo][ind])
                    hiddenform = newlist[ind]
                    #newlist[ind] = '[______]'
                print(i,foo,engsentcits[foo],end='\t',sep='\t',file=outf)
#print(i,engsentcits[foo],len(greeksentlems[foo]),foo,greeksentforms[foo][ind],greeksentpostags[foo][ind],greeksentrels[foo][ind])
                print(' '.join(newlist),engsents[foo],end='\t',sep='\t',file=outf)
                #print(engsents[foo],file=outf)
                #print(' '.join(greeksentlems[foo]))
                #print(' '.join(greeksentlems[foo]))
                print(interl,len(answers),answers,sentrels,file=outf,sep='\t')
                print()
    print('done' )      
    outf.close()       
fname = curtb

getgreeksents(curtb)
sword = 'στρατηγός'
sword = 'ἔχει'
sword = 'Δαρεικός'
sword = 'ἦν'
sword = 'ἦσαν'
sword = 'ἔχει'
sword = 'πέμπουσι'
sword = 'στρατηγός'
sword = 'ἀδελφός'
sword = 'ποταμός'
sword = 'κώμη'
#if(curtb == anabasistb):
#    searchvoc(sword)
#searchvoc(['κώμη','σκηνή','σπονδή','μάχη','φυγή','φυλακή'],[])
l07n = ['ἀγορά','ἀρχή','ἡμέρα','ἐπιτήδειος','στρατιά','φιλία','χώρα']
#searchvoc(['φεύγω','φυλάσσω','παὐω'],['pia'])
#searchvoc(l07n,[])
verblist = ['ἀθροίζω','ἁρπάζω','φεύγω','γράφω','ἐθέλω','λύω','παύω']
vbslesson09 = ['ἄρχω','μέλλω','διώκω','νομίζω','πείθω','ὑποπτεύω']
#searchvoc(verblist+vbslesson09,['iia','pia','pna','fia','fna'])
#searchvoc(['ἀρχαῖος','βάρβαρος','ἔργον','κραυγή','σιγή','θύρα','ὁδός','ἰσχυρός'],[])
#searchvoc(['ἄρχω'],['pia','iia','fia'])


len 3692


In [23]:
optatives = []
perfects = []
aoristpass = []
comparatives = []
superlatives = []
mediopassives = []
miverbs = []
genabsolutes = []
allverbs = {}
totforms = 0

def getfeats(curid):
    curform = ''
    if(curid in formdict):
        curform = formdict[curid]
    #print('gf',curid,curform)
    
    curpos = ''
    m = re.search('postag="([^"]+)"',curform)
    if(m):
        curpos = m[1]

    curlemma = ''
    m = re.search('lemma="([^"]+)"',curform)
    if(m):
        curlemma = m[1]

    curhead = ''
    m = re.search('head="([^"]+)"',curform)
    if(m):
        curhead = m[1]

    curinfl = ''
    m = re.search('form="([^"]+)"',curform)
    if(m):
        curinfl = m[1]

    currel = ''
    m = re.search('relation="([^"]+)"',curform)
    if(m):
        currel = m[1]
    return(curpos,curlemma,curinfl,curhead,currel)

preplist = {}
prepusages = []
prep_acc = {}
prep_gen = {}
prep_dat = {}
def prepdep(curid,curhead):
    #print(curid,curhead)
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    if(headlemma == '(καί|,)' or re.search('^[bu]',headpos)):
        #print('kai')
        curhead = headhead
        headpos,headlemma,headform,headhead,headrel = getfeats(curhead)

    if(curlemma == '(καί|,)' or re.search('^[bu]',curpos)):
        return

    if(re.search('^r',headpos)):
        m = re.search('([a-z].)$',curpos)
        if(m):
            curcase = m[1]
        else:
            curcase = 'x'
        #print(headlemma,curcase,curform,curlemma,curpos,citdict[curid])
        additem(preplist,headlemma)
        prepusages.append([headlemma,curcase,curform,curlemma,curpos,citdict[curid]])
        if(curcase[0] == 'a'):
            additem(prep_acc,headlemma)
        elif(curcase[0] == 'g'):
            additem(prep_gen,headlemma)
        elif(curcase[0] == 'd'):
            additem(prep_dat,headlemma)
noungennouns = []

def getnoungennouns(curid):
    global noungennouns
    curcit = citdict[curid]
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    
    #if(re.search('d-$',curpos) and re.search('ADV',currel) and re.search('^v',headpos)):
    if(re.search('^n.+g-$',curpos) and currel == 'ATR' and re.search('^[pn]',headpos)):
        noungennouns.append([len(noungennouns)+1,curform,currel,headform,headlemma,curid,citdict[curid]])
        #print('ngn',len(noungennouns),curform,currel,headform,headlemma,curid,citdict[curid])

adj2dats = []

def getadj2dats(curid):
    global adj2dats
    curcit = citdict[curid]
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    
    #if(re.search('d-$',curpos) and re.search('ADV',currel) and re.search('^v',headpos)):
    if(re.search('d-$',curpos) and currel == 'ADV' and re.search('^a',headpos)):
        adj2dats.append([len(adj2dats)+1,curform,currel,headform,headlemma,curid,citdict[curid]])
        #print('a2d',len(adj2dats),curform,currel,headform,headlemma,curid,citdict[curid])


def hasarticle(curid):
    for foo in hasdeps[curid]:
        if(foo == '0'):
            continue
        if(re.search('postag="l',formdict[foo])):
            return(foo)
    return('')
            
def autos(curid):
    curcit = citdict[curid]
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    gnc1 = ''
    gnc2 = ''
    m = re.search('^..([sdp]...([mnfc][nvadg]))',curpos)
    if(m):
        gnc1 = m[1] + m[2]
        
    m = re.search('^..([sdp]...([mnfc][nvadg]))',headpos)
    if(m):
        gnc2 = m[1] + m[2]
    art = hasarticle(curhead)
    if(gnc1 and gnc1 == gnc2 and curlemma == 'αὐτός'):
        if(art):
            artn = int(art)
            autosn = int(curid)
            headn = int(curhead)
            # autos o Kuros
            return
            if( artn > autosn and artn < headn ):
                print('\nintensive',formdict[art])
            # o autos  Kuros
            elif( artn < autosn and artn < headn ):
                print('\nthesame',formdict[art])
            else:
                print('\nnocat'+formdict[art])
        #print(int(curhead)-int(curid),curlemma,curform,headform,headlemma,curpos,headpos,curcit)
        
dmeans = 0
def datmeans(curid):
    global dmeans
    curcit = citdict[curid]
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    
    #if(re.search('d-$',curpos) and re.search('ADV',currel) and re.search('^v',headpos)):
    if(re.search('d-$',curpos) and currel == 'ADV' and re.search('^v',headpos)):
        dmeans = dmeans + 1
        #print('dmeans',dmeans,curform,currel,headform,curid,citdict[curid])
        
iobjverbs = {}       
iobj = 0
def iobjs(curid):
    global iobj
    curcit = citdict[curid]
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    
    #if(re.search('d-$',curpos) and re.search('ADV',currel) and re.search('^v',headpos)):
    if(re.search('d-$',curpos) and currel == 'OBJ' and re.search('^v',headpos)):
        iobj = iobj + 1
        if(not headlemma in iobjverbs):
            iobjverbs[headlemma] = {}
        curdict = iobjverbs[headlemma]
        additem(curdict,curlemma)
        #print('iobj',iobj,curform,currel,headform,headlemma,curid,citdict[curid])

conditionals = {}

def getconditional(curid):
    curcit = citdict[curid]
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    headpos2,headlemma2,headform2,headhead2,headrel2 = getfeats(headhead)
    
    
    if(curpos[0] == 'v' and re.search('^(εἰ|ἐάν)$',headlemma)):
        print('cond',curcit,curid,headform,curform,headform2)
    
    
gobjverbs = {}       
gobj = 0
def gobjs(curid):
    global gobj
    curcit = citdict[curid]
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    
    #if(re.search('d-$',curpos) and re.search('ADV',currel) and re.search('^v',headpos)):
    if(re.search('g-$',curpos) and currel == 'OBJ' and re.search('^v',headpos)):
        gobj = gobj + 1
        if(not headlemma in gobjverbs):
            gobjverbs[headlemma] = {}
        curdict = gobjverbs[headlemma]
        additem(curdict,curlemma)
        #print('gobj',gobj,curform,currel,headform,headlemma,curid,citdict[curid])

verbarg = 0
def verbargs(curid,curverb,curcase,needrel):
    global verbarg
    subjform = 'nosbj'
    subjlemma = 'nosbj'
    curcit = citdict[curid]
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    
    #if(re.search('d-$',curpos) and re.search('ADV',currel) and re.search('^v',headpos)):
    if(needrel and not currel == needrel):
        return
    if(re.search(curcase + '-$',curpos) and headlemma == curverb):
        verbarg = verbarg + 1
        for foo in hasdeps[curhead]:
            deppos,deplemma,depform,dephead,deprel = getfeats(foo)
            if(deprel == 'SBJ'):
                subjform = depform
                subjlemma = deplemma
        print('heads',verbarg,curform,currel,headform,headlemma,subjform,subjlemma,curid,citdict[curid])

        
aobjverbs = {} 
aobjnoms = {}
aobj = 0
def aobjs(curid):
    global aobj
    curcit = citdict[curid]
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    
    #if(re.search('d-$',curpos) and re.search('ADV',currel) and re.search('^v',headpos)):
    if(re.search('a-$',curpos) and currel == 'OBJ' and re.search('^v',headpos)):
        aobj = aobj + 1
        if(not headlemma in aobjverbs):
            aobjverbs[headlemma] = {}
        curdict = aobjverbs[headlemma]
        additem(curdict,curlemma)
        
        if(not curlemma in aobjnoms):
            aobjnoms[curlemma] = {}
        curdict = aobjnoms[curlemma]
        additem(curdict,headlemma)
            #aobjverbs[headlemma] = 1
        #print('aobj',aobj,curform,currel,headform,headlemma,curid,citdict[curid])

subj2verbs = {}        
def getsubj2verbs(curid):
    global subj2verbs
    curcit = citdict[curid]
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    
    if(not currel == 'SBJ'):
        return
    lemkey = headlemma + '-' + headpos[5]
    if(not curlemma in subj2verbs):
        subj2verbs[curlemma] = {}
    additem(subj2verbs[curlemma],lemkey)
    
    
    
doubleobjs = {}
hasobjs = {}
def finddoubleobjs(curid):
    global doubleobjs
    curcit = citdict[curid]
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    
    #if(re.search('d-$',curpos) and re.search('ADV',currel) and re.search('^v',headpos)):
    if(re.search('a-$',curpos) and currel == 'OBJ' and re.search('^v',headpos)):
        curkey = curid + ' ' + curform + ' ' + curlemma
        if(curhead in hasobjs):
            hasobjs[curhead].append(curkey)
            #print('multobj',citdict[curid],len(hasobjs[curhead]),headform,headlemma,curhead,hasobjs[curhead])
        else:
            hasobjs[curhead] = [curkey]
            
            


for foo in formdict:
    if(not re.search('^1\.',citdict[foo]) and 0):
        continue
    curpos,curlemma,curform,curhead,currel = getfeats(foo)
    if(curpos[0] == 'v'):
        if(curlemma in allverbs):
            allverbs[curlemma] = allverbs[curlemma] + 1
        else:
            allverbs[curlemma] = 1

    totforms = totforms + 1
    curf = formdict[foo]
        
    if(re.search('"[^"]+g-".+SBJ',curf)):
        genabsolutes.append(curf)
    if(re.search('"v...o',curf)):
        optatives.append(curf)
    if(re.search('"v..[rl].',curf)):
        perfects.append(curf)
    if(re.search('"v..a.p',curf)):
        aoristpass.append(curf)
    elif(re.search('"v....[pme]',curf)):
        mediopassives.append(curf)
    if(re.search('c" head',curf)):
        comparatives.append(curf)
    if(re.search('lemma="[^"]+μι"',curf)):
        miverbs.append(curf)
    if(re.search('postag="[^"]+s"',curf)):
        superlatives.append(curf)
    m = re.search('head="([^"]+)"',curf)
    if(m):
        curhead = m[1]
    m = re.search('id="([^"]+)"',curf)
    if(m):
        curid = m[1]
    
    prepdep(curid,curhead)
    autos(curid)
    datmeans(curid)
    aobjs(curid)
    gobjs(curid)
    iobjs(curid)
    verbargs(curid,'περί','[d]','')
    getnoungennouns(curid)
    getadj2dats(curid)
    finddoubleobjs(curid)
    getsubj2verbs(curid)
    getconditional(curid)

i = 0

def cntdict(curdict1,curkey):
    if(curkey in curdict1):
        curdict = curdict1[curkey]
    else:
        return(0,0)
    dictlen = len(curdict)
    dicttot = 0
    #print(dictlen,curdict)
    for boo in curdict:
        dicttot = dicttot + curdict[boo]
    return(dictlen,dicttot)

for foo in sorted(allverbs,key=allverbs.get,reverse=True):
    i = i + 1
    dats,dats2 = cntdict(iobjverbs,foo)
    accs,accs2 = cntdict(aobjverbs,foo)
    gens,gens2 = cntdict(gobjverbs,foo)

    
    print(i,allverbs[foo],foo,accs,dats,gens)
#for foo in sorted(preplist):
#    print(foo)
print(totforms)
print('mediopassives',len(optatives))
print('optative',len(optatives))
print('aoristpassive',len(aoristpass))
print('comparatives',len(comparatives))
print('superlatives',len(superlatives))
print('perfects',len(perfects))
print('miverbs',len(miverbs))

   
    

cond 1.1.4 8996139 ἢν δύνηται βασιλεύσει
cond 1.2.2 8996718 εἰ καταπράξειεν παύσεσθαι
cond 1.2.21 8997793 εἴ ἐκώλυεν εἰσελθεῖν
cond 1.2.27 8998188 ἤν ἐντυγχάνωσιν ἀπολαμβάνειν
cond 1.3.4 8998383 εἴ δέοιτο ὠφελοίην
cond 1.3.5 8998422 εἰ ποιήσω οἶδα
cond 1.3.6 8998501 ἂν ὦ εἶναι
cond 1.3.11 8998758 εἴ δοκεῖ μενοῦμεν
cond 1.3.14 8998896 εἰ βούλεται ἑλέσθαι
cond 1.3.14 8998929 ἐὰν διδῷ αἰτεῖν
cond 1.3.14 8998946 ἐὰν διδῷ συντάττεσθαι
cond 1.3.16 8999076 εἰ πιστεύσομεν κωλύει
cond 1.3.18 8999174 ἐὰν ᾖ ἕπεσθαι
cond 1.3.19 8999202 ἐὰν φαίνηται ἀξιοῦν
cond 1.3.20 8999308 ἢν φύγῃ βουλευσόμεθα
cond 1.4.5 8999661 εἰ φυλάττοιεν βιασόμενος
cond 1.4.7 8999820 εἰ ἁλώσοιντο ᾤκτιρον
cond 1.4.9 8999951 εἴ ἦν συνεπορεύοντο
cond 1.4.12 9000158 ἐὰν διδῷ ἰέναι
cond 1.4.14 9000260 ἐάν πεισθῆτε προτιμήσεσθε
cond 1.4.15 9000311 ἢν ψηφίσωνται δόξετε
cond 1.4.15 9000342 ἢν ἀποψηφίσωνται ἄπιμεν
cond 1.5.1 9000595 εἰ ἐνῆν ἦσαν
cond 1.5.2 9000682 εἰ θηρῷεν ἦν
cond 1.5.3 9000736 ἄν ἀνιστῇ ἔστι
heads 1 τραχήλοις ATR 

cond 4.7.3 9029056 εἰ ληψόμεθα ἔστι
cond 4.7.5 9029123 ἢν ἀναλώσωσιν κωλύει
cond 4.7.5 9029154 εἰ ὡπλισμένους ὁρῶμεν
cond 4.7.7 9029259 ἢν δυνώμεθα ἔσται
cond 4.7.7 9029266 ἢν βουλώμεθα ἀπελθεῖν
cond 4.8.4 9030242 εἰ κωλύει ἐθέλω
cond 4.8.7 9030322 εἰ δοῖεν ἠρώτων
cond 4.8.11 9030498 ἢν προσάγωμεν περιττεύσουσιν
cond 4.8.11 9030518 ἐὰν ὦμεν εἴη
cond 4.8.11 9030525 εἰ διακοπείη εἴη
cond 4.8.11 9030542 εἰ ἔσται ἔσται
cond 4.8.12 9030599 ἂν ᾖ ἄξει
cond 4.8.13 9030636 ἄν πιέζηται βοηθήσει
cond 4.8.13 9030648 ἤν δυνηθῇ μείνῃ
cond 4.8.14 9030706 ἤν δυνώμεθα δεῖ
cond 5.1.4 9031486 ἢν πέμψητέ οἴομαι
cond 5.1.4 9031511 ἂν ἔλθω ἔστ’
cond 5.1.6 9031602 ἢν πορεύησθε ἀπόλλυσθαι
cond 5.1.8 9031675 ἐάν δέῃ εἰδῶμεν
cond 5.1.8 9031693 ἐάν ἐγχειρῇ ξυμβουλεύωμεν
cond 5.1.9 9031751 ἐὰν φυλάττωμεν δύναιντο
cond 5.1.10 9031770 εἰ ἠπιστάμεθα ἔδει
cond 5.1.10 9031803 ἢν ἔλθῃ πλευσόμεθα
cond 5.1.10 9031814 ἢν ἄγῃ χρησόμεθα
cond 5.1.11 9031834 εἰ κατάγοιμεν ἀπορήσαιμεν
cond 5.1.12 9031868 εἰ εἰκὸς ἐννοήσατε
con

cond 7.4.1 9053818 ἂν πείθωνται ἀπῄει
heads 3 στέρνοις ADV περὶ περί nosbj nosbj 9053916 7.4.4
heads 4 μηροῖς CO περὶ περί nosbj nosbj 9053921 7.4.4
cond 7.4.5 9053950 εἰ καταβήσονται ὅτι
cond 7.4.8 9054090 εἴ εἶεν ἢ
cond 7.4.9 9054128 εἰ κελεύει παῖε
cond 7.4.10 9054142 εἰ παίσειεν ἐπήρετο
cond 7.4.20 9054659 εἰ βούλεται συστρατεύεσθαι
cond 7.4.23 9054772 εἰ βούλοιτο σπείσασθαι
cond 7.4.24 9054794 εἰ ἔσονται ἔχειν
cond 7.5.5 9054990 εἰ ἐκήδου ἧκες
cond 7.5.5 9055004 εἰ ἐδύνω προσδανεισάμενος
cond 7.6.5 9055650 ἢν ὑπόσχησθε ἀποδραμοῦνται
cond 7.6.7 9055736 ἢν ἴητε τιμωρήσεσθε
cond 7.6.9 9055828 εἰ ἀπήγαγεν ἦμεν
cond 7.6.10 9055874 εἰ ἴδοιμι λέγων
cond 7.6.11 9055969 εἴ δυναίμην ὠφελήσων
cond 7.6.12 9055988 εἰ πείσαιμι ὑπισχνουμένου
cond 7.6.15 9056111 εἰ ἐπαινῶ αἰτιῷσθε
cond 7.6.15 9056133 εἰ εἰμι ἔχοιμι
cond 7.6.16 9056197 εἰ ἐδίδου ἐδίδου
cond 7.6.17 9056217 εἰ οἴεσθε ἔξεστιν
cond 7.6.17 9056232 ἐὰν πράττητε ποιῆσαι
cond 7.6.17 9056243 εἰ ἔχω ἀπαιτήσει
cond 7.6.17 9056258 ἐὰν βεβαιῶ 

426 4 περίειμι 0 0 1
427 4 στέρομαι 0 0 2
428 4 ὑπηρετέω 2 3 0
429 4 κόπτω 3 1 0
430 4 ἐκλέγω 2 0 2
431 4 γελάω 0 0 0
432 4 ὑπάγω 1 0 0
433 4 φρονέω 0 0 0
434 4 ἐπινοέω 2 0 0
435 4 θωρακίζω 0 0 0
436 4 νέμω 1 1 0
437 4 ζητέω 3 0 0
438 4 ἐξαιρέω 3 1 0
439 4 ἐργάζομαι 3 0 0
440 4 μαίνομαι 1 0 0
441 4 φιλοφρονέομαι 2 0 0
442 4 ἀνακοινόω 0 3 0
443 4 συμπροθυμέομαι 1 0 0
444 4 ἔργω 2 0 0
445 4 ὑφίημι 0 0 0
446 4 ἐξορμάω 0 1 0
447 4 προσίημι 2 1 0
448 4 περαίνω 2 0 0
449 4 στέλλω 1 0 0
450 4 καταθύω 2 1 1
451 4 μεταδίδωμι 2 2 0
452 4 ἀριστοποιέω 0 1 0
453 4 κατακλείω 2 0 0
454 4 ἀναχωρέω 0 0 0
455 4 ἀποκόπτω 3 0 0
456 4 διακελεύομαι 0 3 1
457 4 ἀποτρέπω 1 0 0
458 4 ὁρμίζω 0 0 0
459 4 φάσκω 0 0 0
460 4 περιέρχομαι 0 0 0
461 4 κυλίνδω 2 0 0
462 4 συνομολογέω 1 2 0
463 4 ὁρίζω 3 0 1
464 4 ἀποσκεδάννυμι 1 0 0
465 4 σφαγιάζομαι 0 1 0
466 4 πίνω 2 0 0
467 4 προάγω 1 0 0
468 4 ἀναλίσκω 2 0 0
469 4 ἐπιλαμβάνω 1 0 1
470 4 διαφεύγω 0 0 0
471 4 ἐνάπτω 1 0 0
472 4 οἰκίζω 2 0 0
473 4 μετέχω 0 0 2
474 4 κ

926 1 καταλέγω 1 0 0
927 1 γενειάω 0 0 0
928 1 διείργω 0 0 0
929 1 ἀνακαίω 1 0 0
930 1 συμβαίνω 0 0 0
931 1 ἐπιδίδωμι 1 0 0
932 1 ἐκφαίνω 1 0 0
933 1 ἀντεπιμελέομαι 1 0 0
934 1 ἀνασταυρόω 0 0 0
935 1 μακαρίζω 1 0 0
936 1 διαθεάομαι 1 0 1
937 1 μέτειμι 0 1 1
938 1 ἐνθυμόω 1 0 0
939 1 προφασίζομαι 1 0 0
940 1 ἀκμάζω 0 0 0
941 1 ἐρύκω 1 0 0
942 1 βοιωτιάζω 0 0 0
943 1 παρασκηνέω 0 1 0
944 1 κεντέω 0 0 0
945 1 ἐράω 0 0 1
946 1 τρυπάω 1 0 0
947 1 προβουλεύω 0 0 1
948 1 προπονέω 0 0 0
949 1 ἀντικαθίστημι 0 0 0
950 1 συναιρέω 0 0 0
951 1 ἐγχειρίζω 1 1 0
952 1 πτάρνυμαι 0 0 0
953 1 συνεπεύχομαι 0 0 0
954 1 ἐμπεδέω 1 0 0
955 1 ἀγείρω 1 0 0
956 1 λακτίζω 0 0 0
957 1 βαίνω 0 0 0
958 1 προσιόω 0 0 0
959 1 καρπέω 1 0 0
960 1 βιοτεύω 0 0 0
961 1 ὁμιλέω 0 2 1
962 1 ἐπιλανθάνω 0 0 1
963 1 πένομαι 0 0 0
964 1 πολιτεύω 0 0 0
965 1 συγκατακαίω 1 0 0
966 1 συνωφελέω 1 0 0
967 1 ἐπικυρόω 0 0 0
968 1 προσδοκέω 0 1 1
969 1 καίνω 0 0 0
970 1 διαπολεμέω 0 1 0
971 1 ἀντιποιόω 1 0 0
972 1 ἐπαναχωρέω 1 0 0
973 1 

In [19]:

for foo in subj2verbs:
    curdict = subj2verbs[foo]
    print('\n',foo)
    for boo in sorted(curdict,key=curdict.get,reverse=True):
        print(curdict[boo],boo)
        


 παῖς
1 γίγνομαι-e
1 πάρειμι-a
1 παιδεύω-e
1 θεάομαι-e
1 ἀγωνίζομαι-e
1 ποιέω-e
1 κελεύω-a
1 ἐάω-a

 Δαρεῖος
1 ἀσθενέω-a
1 τελευτάω-a

 πρέσβυς
1 τυγχάνω-a
1 προσλαμβάνω-a
1 ἔρχομαι-e
1 κελεύω-a
1 οἴχομαι-e
1 πέμπω-a
1 προβάλλω-e

 ὅσος
8 εἰμί-a
2 τυγχάνω-a
1 ἀθροίζω-e
1 διαλέγω-e
1 πράσσω-p
1 συνέρχομαι-a
1 γίγνομαι-m
1 πολεμέω-p
1 ἀφικνέομαι-m
1 ἐμπίμπρημι-a
1 δύναμαι-p
1 κοιμάω-e
1 ἐσθίω-a
1 καταφεύγω-a
1 εἰμί-m

 Κῦρος
6 λέγω-a
4 ἀναβαίνω-a
4 δίδωμι-a
4 φημί-a
4 τελευτάω-a
3 ἥδομαι-p
3 ἐξελαύνω-a
3 ἐρωτάω-a
3 ἀποθνῄσκω-a
2 ἀποπέμπω-a
2 μένω-a
2 θεωρέω-a
2 μεταπέμπω-e
2 γίγνομαι-m
2 ποιέω-e
2 κελεύω-a
2 εἰμί-a
2 δοκέω-a
2 πέμπω-a
2 θνῄσκω-a
1 πολιορκέω-a
1 ἄγαμαι-p
1 ὁρμάω-e
1 ποιέω-a
1 ἀποδίδωμι-a
1 συγγίγνομαι-m
1 ἀποκτείνω-a
1 καλέω-a
1 αἰτέω-a
1 φθάνω-a
1 ἀέκων--
1 ὑπισχνέομαι-e
1 μεταπέμπω-m
1 οἴομαι-e
1 ἐάω-a
1 διώκω-a
1 ἐκκόπτω-a
1 δέω-e
1 οἶδα-m
1 διαβαίνω-a
1 ἐφίστημι-a
1 ἥκω-a
1 ἐπέρχομαι-a
1 συλλαμβάνω-a
1 ἄρχω-a
1 συμβουλεύω-e
1 προσελαύνω-a
1 παρέρχομαι-a
1 πορεύω-e
1 

2 εἰμί-a
1 λέγω-e

 εὖρος
5 εἰμί-a

 Ἀπόλλων
1 λέγω-e
1 ἀναιρέω-a

 Ξέρξης
1 λέγω-e
1 ἔρχομαι-a

 Σῶσις
1 πάρειμι-a

 ὁπλίτης
2 γίγνομαι-m
2 τίθημι-m
1 ἔρχομαι-a
1 μένω-a
1 ἐκθλίβω-e
1 ἥκω-a
1 θέω-a
1 εἰμί-a
1 ἕπομαι-e
1 προσθέω-a
1 ἡγέομαι-e
1 τρέχω-m

 ἆθλον
1 εἰμί-a
1 κεῖμαι-e

 μισθός
2 φαίνω-e
1 ὀφείλω-e
1 γίγνομαι-m
1 πάρειμι-m
1 εἰμί-m

 ἀποδίδωμι
2 εἰμί-a

 Ἐπύαξα
1 ἀφικνέομαι-e
1 ἀφικνέομαι-m

 Κίλισσα
1 ἔχω-a
1 λέγω-e
1 παρελαύνω-a
1 φεύγω-a
1 θαυμάζω-a

 συγγίγνομαι
1 λέγω-e
1 δοκέω-a

 κρήνη
1 εἰμί-a

 Μίδας
1 λέγω-e

 Σάτυρος
1 θηρεύω-a

 θηρεύω
1 λέγω-e

 Ἕλλην
7 εἰμί-a
5 ὁράω-a
3 διώκω-a
3 νικάω-a
3 πράσσω-a
3 στρατοπεδεύω-e
2 ἔρχομαι-a
2 ἀποκρίνω-m
2 ἵστημι-a
2 χωρέω-a
2 ὄμνυμι-a
2 μένω-a
2 θέω-a
2 ἀπέρχομαι-a
2 ἀφικνέομαι-m
2 γιγνώσκω-a
2 ἔχω-a
2 παρέρχομαι-a
2 φεύγω-a
2 ἀδικέω-a
2 πορεύω-p
1 τάσσω-p
1 συμπορεύομαι-e
1 δέω-e
1 δοκέω-a
1 παιανίζω-a
1 αἰσθάνομαι-m
1 παρασκευάζω-e
1 παίω-a
1 δείδω-a
1 κοιμάω-p
1 ἀναγκάζω-a
1 εὑρίσκω-e
1 ἀκούω-a
1 παραδίδωμι-a
1 τίθημι-e
1

1 ζάω-a
1 πλήσσω-a
1 πάρειμι-a
1 ἕπομαι-e
1 εἰσέρχομαι-a
1 δημαγωγέω-a
1 ὑπισχνέομαι-e

 ἀπορία
3 εἰμί-a

 μένω
3 εἰμί-a
1 εἰμί-m

 εἷς
2 εἰμί-a
1 λέγω-a
1 δύναμαι-p
1 μένω-a
1 ἐξάρχω-a
1 καταλείπω-e

 ἐπιτήδειος
5 εἰμί-a
1 ἀγοράζω-e
1 ἐπιμελέομαι-m
1 δέω-e

 ἀγορά
2 εἰμί-a
1 πάρειμι-a
1 ἀφικνέομαι-e

 μηδείς
1 λέγω-a
1 εἰμί-a
1 μετέχω-a
1 λανθάνω-a
1 φθέγγομαι-e
1 τελέω-a
1 εἰσέρχομαι-a

 αἰτέω
1 εἰμί-a
1 δεῖ-a

 κελεύω
1 κωλύω-a

 ἐξέρχομαι
1 εἰμί-m
1 εἰμί-a
1 δεῖ-a
1 λαμβάνω-a

 ἐρωτάω
1 δοκέω-a

 πρᾶξις
2 εἰμί-a
1 φαίνω-e

 φίλος
1 ἕπομαι-e
1 τυγχάνω-m
1 σατραπεύω-a
1 ποιέω-a
1 ἐμβάλλω-a
1 ἀποθνῄσκω-a
1 λείπω-m
1 εἰμί-a

 αἱρετός
1 ἀγγέλλω-a

 ὑποψία
2 εἰμί-a

 ναῦς
1 πάρειμι-a
1 ὁρμάω-a

 Ταμῶς
1 ἡγέομαι-e

 Χειρίσοφος
8 λέγω-a
4 ἡγέομαι-e
3 φημί-a
3 πορεύω-e
2 πάρειμι-a
2 ἀνίστημι-a
2 πέμπω-a
2 ἀφικνέομαι-e
2 ἥκω-a
1 τυγχάνω-a
1 αἰτιάομαι-e
1 ὁράω-a
1 συμπέμπω-a
1 ἀναβαίνω-a
1 ὑπομένω-a
1 ἀποκρίνω-e
1 λύω-a
1 ποιέω-a
1 λαμβάνω-a
1 ἐμβαίνω-a
1 διώκω-a
1 κατέχω-a
1 στρατοπεδεύω-m
1

1 διασπάω-p
1 διακόπτω-p
1 ὑπαντιάζω-a

 οἷος
2 εἰμί-a
1 ἐλελίζω-a

 ἅρμα
1 φέρω-e

 καταλαμβάνω
1 εἰμί-a

 τοξεύω
1 λέγω-e

 ἰσχύς
1 εἰμί-a

 τροπή
1 γίγνομαι-m
1 γίγνομαι-e

 ἑξακόσιοι
1 διασπείρω-e

 ὀλίγος
2 ἀποθνῄσκω-a
1 καταλείπω-p
1 γεύω-m
1 ἀνακαίω-a
1 φαίνω-e
1 δύναμαι-e

 Κτησίας
1 φημί-a
1 ἀποθνῄσκω-a

 ὁπόσος
3 εἰμί-a
1 ἀποθνῄσκω-a
1 γίγνομαι-m
1 μαστεύω-a
1 γιγνώσκω-a
1 πρόσειμι-a

 Ἀρταπάτης
1 λέγω-e

 ἀκούω
1 εἰμί-a
1 δοκέω-a

 κακοῦργος
1 καταγελάω-a

 ὁράω
6 εἰμί-a
4 ἔξεστι-a

 πορεύω
4 δεῖ-a
2 εἰμί-a
2 χρή-a
1 γίγνομαι-m
1 ἀσφαλής--
1 ἔξεστι-a

 πόλεμος
5 εἰμί-a
2 ἀπιόω-a
2 γίγνομαι-m
1 προιόω-a

 ἀφθονία
2 εἰμί-a

 πειθαρχέω
1 εἰμί-a

 ὑπηρέτης
1 λέγω-p

 εὔνους
1 εἰμί-a

 τυγχάνω
2 εἰμί-a
1 τε--
1 δεῖ-a

 γίγνομαι
5 εἰμί-a
1 ὁμολογέω-e
1 τάσσω-p
1 δεῖ-a
1 διασπάω-e
1 δηλόω-e
1 ἔξεστι-a
1 πάρειμι-a
1 πάσχω-a
1 πείθω-p

 θεραπεύω
1 ὁμολογέω-e

 σῶμα
1 κοσμέω-p

 φέρω
1 ἐπιλέγω-a

 χιλός
1 εἰμί-a

 ἕ
1 εἰμί-a
1 κελεύω-a

 κεφαλή
1 ἀποτέμνω-e

 παρασάγγης
2 λέγω-e

 Μιλ

1 γίγνομαι-e

 ἀποβάλλω
1 εἰμί-a

 διακρίνω
1 δοκέω-a

 ἀνακοινόω
1 εἰμί-a

 ὄρνεον
1 ἐπιτίθημι-e

 ἀετός
1 λαμβάνω-a

 πέτομαι
1 λαμβάνω-a

 προκριθάω
1 δοκέω-a

 στάσις
1 εἰμί-a

 σύνδειπνος
1 αἱρέω-e

 λοχαγέω
1 ἔξεστι-a

 ἐπιτρέπω
1 εἰμί-a

 ἀπέχω
2 δεῖ-a

 Δέξιππος
2 διαβάλλω-a
1 βάλλω-a
1 λέγω-a
1 εἰμί-a

 Ἀργώ
1 λέγω-e

 Ἡρακλῆς
1 λέγω-e

 σημεῖον
1 δείκνυμι-a

 Λύκων
1 λέγω-a
1 ἐπαπειλέω-a

 ξένιος
1 γίγνομαι-m

 ταράσσω
1 αἰτιάομαι-e

 Ἀρκάς
3 εἰμί-a
1 συνίστημι-e
1 πλέω-a
1 πορεύω-e
1 πυνθάνομαι-e
1 ἕπομαι-e
1 ἐάω-a
1 λέγω-a

 κατεργάζομαι
1 εἰμί-a

 Κλέανδρος
3 φημί-a
2 λέγω-a
1 μέλλω-a
1 ἀφικνέομαι-e
1 φεύγω-a
1 ἀπέρχομαι-a
1 ἀκούω-a
1 θύω-e
1 πιπράσκω-a

 φάος
1 γίγνομαι-m

 ἁλίζω
1 δεῖ-a

 διαφεύγω
1 ἀθροίζω-e

 Ἡγήσανδρος
1 σῴζω-p

 πολιορκέω
1 ἀπόλλυμι-m

 διακινδυνεύω
1 εἰμί-a

 Θρᾳκία
1 εἰμί-a

 λιμήν
1 κεῖμαι-e

 ἐξάγω
1 εἰμί-a
1 τελέθω-a

 Ἀρηξίων
1 πάρειμι-a
1 σφαγιάζομαι-e

 θῦμα
1 ἐπιλείπω-a

 πρόβατον
2 εἰμί-a

 ἱερεῖον
1 εἰμί-a
1 ἵστημι-a

 εἴσοδος
1 εἰμί-a

 ἀ

In [7]:
curdict = aobjnoms['στράτευμα']
i = 0
print(len(curdict))
for foo in sorted(curdict,key=curdict.get,reverse=True):
    i = i + 1
    print(i,foo,curdict[foo])
    


30
1 ἔχω 6
2 συλλέγω 4
3 ἄγω 3
4 διαβαίνω 2
5 συνάγω 2
6 προάγω 2
7 ποιέω 2
8 ὁράω 2
9 ἀθροίζω 1
10 λαμβάνω 1
11 ἀποπέμπω 1
12 ἐπιδείκνυμι 1
13 εὑρίσκω 1
14 ἐπιλείπω 1
15 συναγείρω 1
16 διαγγέλλω 1
17 ἵστημι 1
18 καθίστημι 1
19 ἐφίστημι 1
20 ὠφελέω 1
21 μισέω 1
22 φοβέω 1
23 διαβιβάζω 1
24 διαφθείρω 1
25 συγκαλέω 1
26 συνέπομαι 1
27 ἀποδίδωμι 1
28 ἀπάγω 1
29 ἐξάγω 1
30 παραλαμβάνω 1


In [99]:
def prepuse(curprep):
    curacc = 0
    curgen = 0
    curdat = 0
    if( curprep in prep_acc ):
        curacc = prep_acc[curprep]
    if( curprep in prep_gen ):
        curgen = prep_gen[curprep]
    if( curprep in prep_dat ):
        curdat = prep_dat[curprep]
    print(curprep,preplist[curprep],curgen,curdat,curacc)

for foo in sorted(preplist,key=preplist.get,reverse=True):
    prepuse(foo)              

ἐπί 462 70 121 267
ἐν 440 1 435 0
εἰς 436 0 0 409
πρός 313 25 16 264
ἐκ 251 245 0 2
παρά 183 73 37 73
σύν 169 1 168 0
διά 160 92 1 47
κατά 124 9 0 115
περί 123 71 4 45
ὑπό 114 97 8 7
ἀπό 112 112 0 0
μετά 87 28 0 59
ἀμφί 50 1 0 44
ὑπέρ 41 38 0 2
ἕνεκα 37 35 0 1
μέχρι 28 14 0 1
ἀντί 27 26 0 0
πλήν 24 15 0 0
πρό 19 19 0 0
ἄνευ 18 18 0 0
ἀνά 12 0 0 11
ἕως 11 0 0 0
ἐντός 7 7 0 0
μά 5 0 0 5
πέραν 5 4 1 0
ἔμπροσθεν 4 2 0 1
μεταξύ 3 3 0 0
χωρίς 2 1 0 1
νή 2 0 0 2
ἄχρι 2 0 0 0
πλησίον 2 2 0 0
ἀντίον 1 0 0 1
ἐπέκεινα 1 1 0 0
δίχα 1 0 0 0
πέρα 1 1 0 0
ἐναντίον 1 1 0 0


In [38]:
len(depsdict)
i = 0
for foo in depsdict:
    i = i + 1
    print(i,foo,depsdict[foo])
    if(i>20):
        break

i = 0
for foo in formdict:

    headpos = get_postag(formdict[foo])
    curlemma = get_lemma(formdict[foo])
    if(not is_decl1(curlemma)):
        continue
    m = re.search('(.).(.---m.-)',headpos)
    if(m and m[1] == 'n'):
        matchpos = m[2]
        if(not foo in depsdict):
            continue
        matchart = has_matchingarticle(foo,matchpos)
        if(not matchart):
            continue
        for boo in depsdict[foo]:
            if(not re.search('ATR',get_relation(formdict[boo]))):
                continue
            deppos = get_postag(formdict[boo])
            if(not is_decl1(get_lemma(formdict[boo]))):
                continue
            if(re.search('^a',deppos) and re.search(matchpos,deppos)):
                print()
                print(i,citdict[foo])
                matchart2 = has_matchingarticle(boo,matchpos)
                
                if(matchart<boo and boo < foo):
                    i = i + 1
                    if(i>20):
                        break
                    print('attrib1')
                    print(formdict[matchart])
                    print(formdict[boo])
                    print(formdict[foo])
                else:
                    print(formdict[matchart])
                    print(formdict[foo])
                    print(formdict[boo])
    


1 8995997 ['8995993', '8995998']
2 8995995 ['8995994']
3 8995993 ['8995995', '8995999', '8996003', '8996007']
4 8995996 ['8995997', '8996001', '8996002', '8996005']
5 8996002 ['8996000', '8996006']
6 8996006 ['8996004']
7 8996018 ['8996008', '8996009', '8996022']
8 8996008 ['8996010', '8996017', '8996023']
9 8996010 ['8996011', '8996013']
10 8996013 ['8996012', '8996014']
11 8996016 ['8996015']
12 8996014 ['8996016']
13 8996020 ['8996019']
14 8996022 ['8996020', '8996021']
15 8996027 ['8996024']
16 8996029 ['8996025', '8996026', '8996027', '8996028']
17 8996024 ['8996030']
18 8996033 ['8996031', '8996032', '8996034', '8996046']
19 8996036 ['8996035', '8996040']
20 8996034 ['8996036']
21 8996039 ['8996037']

0 1.1.6
    <word id="8996228" form="τοῖς" lemma="ὁ" postag="l-p---md-" head="8996229" relation="ATR"/>
    <word id="8996229" form="φρουράρχοις" lemma="φρούραρχος" postag="n-p---md-" head="8996227" relation="OBJ"/>
    <word id="8996230" form="ἑκάστοις" lemma="ἕκαστος" postag="a-p-

11 2.3.3
attrib1
    <word id="9008789" form="τοῖς" lemma="ὁ" postag="l-p---md-" head="9008791" relation="ATR"/>
    <word id="9008790" form="ἄλλοις" lemma="ἄλλος" postag="a-p---md-" head="9008791" relation="ATR"/>
    <word id="9008791" form="στρατηγοῖς" lemma="στρατηγός" postag="n-p---md-" head="9008794" relation="OBJ"/>

12 2.3.9
attrib1
    <word id="9009017" form="τὸν" lemma="ὁ" postag="l-s---ma-" head="9009019" relation="ATR"/>
    <word id="9009018" form="αὐτὸν" lemma="αὐτός" postag="a-s---ma-" head="9009019" relation="ATR"/>
    <word id="9009019" form="φόβον" lemma="φόβος" postag="n-s---ma-" head="9009020" relation="SBJ"/>

13 2.3.16
    <word id="9009341" form="ὁ" lemma="ὁ" postag="l-s---mn-" head="9009342" relation="ATR"/>
    <word id="9009342" form="ἐγκέφαλος" lemma="ἐγκέφαλος" postag="n-s---mn-" head="9009340" relation="SBJ"/>
    <word id="9009343" form="ὅλος" lemma="ὅλος" postag="a-s---mn-" head="9009342" relation="ATR"/>

13 2.4.2
attrib1
    <word id="9010010" form="τ

35 5.3.10
    <word id="9033844" form="τοῦ" lemma="ὁ" postag="l-s---mg-" head="9033846" relation="ATR"/>
    <word id="9033846" form="χώρου" lemma="χῶρος" postag="n-s---mg-" head="9033842" relation="ATR"/>
    <word id="9033843" form="αὐτοῦ" lemma="αὐτός" postag="a-s---mg-" head="9033846" relation="ATR"/>

35 5.3.10

36 5.3.12
    <word id="9033918" form="τὸν" lemma="ὁ" postag="l-s---ma-" head="9033919" relation="ATR"/>
    <word id="9033919" form="ναὸν" lemma="ναός" postag="n-s---ma-" head="9033915" relation="ADV"/>
    <word id="9033917" form="αὐτὸν" lemma="αὐτός" postag="a-s---ma-" head="9033919" relation="ATR"/>

36 5.4.2
    <word id="9034025" form="τὸν" lemma="ὁ" postag="l-s---ma-" head="9034024" relation="ATR"/>
    <word id="9034024" form="Τιμησίθεον" lemma="Τιμησίθεος" postag="n-s---ma-" head="9034021" relation="OBJ"/>
    <word id="9034026" form="Τραπεζούντιον" lemma="Τραπεζούντιος" postag="a-s---ma-" head="9034024" relation="ATR"/>

36 5.4.27

37 5.4.34

38 5.6.15
    <word 

In [18]:
vparads = ['pia','iia','aia','fia','ria','lia','tia',
           'psa','asa','fsa','rsa','tsa',
           'poa','aoa','foa','roa','toa',
           'pma','ama','fma','rma','tma',
           'ppa','apa','fpa','rpa','tpa',
           'pna','ana','fna','rna','tna',
            
            'pim','iim','aim','fim','rim','lim','tim',
           'psm','asm','fsm','rsm','tsm',
           'pom','aom','fom','rom','tom',
           'pmm','amm','fmm','rmm','tmm',
           'ppm','apm','fpm','rpm','tpm',
           'pnm','anm','fnm','rnm','tnm',
            
            'pip','iip','aip','fip','rip','lip','tip',
           'psp','asp','fsp','rsp','tsp',
           'pop','aop','fop','rop','top',
           'pmp','amp','fmp','rmp','tmp',
           'ppp','app','fpp','rpp','tpp',
           'pnp','anp','fnp','rnp','tnp',
            
            'pie','iie','aie','fie','rie','lie','tie',
           'pse','ase','fse','rse','tse',
           'poe','aoe','foe','roe','toe',
           'pme','ame','fme','rme','tme',
           'ppe','ape','fpe','rpe','tpe',
           'pne','ane','fne','rne','tne',
           ]

tensenames = {
 "p" : "present",
 "i" : "imperfect",
 "r" : "perfect",
 "s" : "resultative",
 "a" : "aorist",
 "u" : "past",
 "l" : "pluperfect",
 "f" : "future",
 "t" : "future perfect",
 "x" : "uncertain tense"
}

moodnames = {
 "i" : "indicative",
 "s" : "subjunctive",
 "m" : "imperative",
 "o" : "optative",
 "n" : "infinitive",
 "p" : "participle",
 "d" : "gerund"
}

voicenames = {
    
 "a" : "active",
 "m" : "middle",
 "p" : "passive",
 "e" : "middle or passive",
 "x" : "unspecified",
}

totvbs  = 0
vparadcnt = {}
def countvparads(curparad):
    global totvbs
    f = open(curtb)
    vpat = 'postag=.v..(...)'
    
    for foo in vparads:
        vparadcnt[foo] = 0
    for l in f:
        m = re.search(vpat,l)
        if(m):
            curv = m[1]
            if(not curv in vparadcnt):
                print(l)
            else:
                vparadcnt[curv] = vparadcnt[curv] + 1
                totvbs = totvbs + 1
        
    
    f.close()

countvparads('pia')
i = 0
sofar = 0
for foo in sorted(vparadcnt,key=vparadcnt.get,reverse=True):
    i = i + 1
    sofar = sofar + vparadcnt[foo]
    print(i,foo,vparadcnt[foo],vparadcnt[foo]/totvbs,sofar/totvbs)

1 iia 1894 0.14248100504024674 0.14248100504024674
2 ppa 1338 0.10065447980139923 0.243135484841646
3 pia 1196 0.0899721658015497 0.33310765064319564
4 pna 1112 0.08365305047769503 0.4167607011208907
5 aia 1084 0.0815466787030768 0.4983073798239675
6 apa 758 0.057022493041450385 0.5553298728654179
7 iie 700 0.0526592943654555 0.6079891672308734
8 ppe 495 0.037237643872714964 0.6452268111035884
9 ana 440 0.03310012788685775 0.678326938990446
10 pie 431 0.032423079816444746 0.7107500188068908
11 pne 400 0.03009102535168886 0.7408410441585797
12 poa 362 0.027232377943278416 0.7680734221018581
13 aim 346 0.02602873692921086 0.794102159031069
14 fim 204 0.015346422929361317 0.8094485819604303
15 rpe 194 0.014594147295569097 0.8240427292559994
16 apm 177 0.01331527871812232 0.8373580079741217
17 psa 168 0.01263823064770932 0.849996238621831
18 aip 152 0.011434589633641766 0.8614308282554728
19 poe 145 0.010907996689987211 0.87233882494546
20 fia 144 0.010832769126607989 0.883171594072068
21 

In [19]:

from tabulate import tabulate
from prettytable import PrettyTable


            
outf = open('parads/test.md','w')

nparad1 = ['s---Zv','s---Zn','s---Zg','s---Zd','s---Za',
           'p---Zv','p---Zn','p---Zg','p---Zd','p---Za']
nparads = ['s---mv','s---mn','s---mg','s---md','s---ma',
            'p---mv','p---mn','p---mg','p---md','p---ma',
            's---fv','s---fn','s---fg','s---fd','s---fa',
            'p---fv','p---fn','p---fg','p---fd','p---fa',
            's---nv','s---nn','s---ng','s---nd','s---na',
            'p---nv','p---nn','p---ng','p---nd','p---na']

def printnounrow(useparad,curparad,curparadf,index,outf):
    if(re.search(',',curparad[useparad[index]])):
        rowvalue1 = curparad[useparad[index]]
    else:
        rowvalue1 = re.sub('\s+[0-9]+$','',curparad[useparad[index]])
    if(re.search(',',curparad[useparad[index+5]])):
        rowvalue2 = curparad[useparad[index+5]]
    else:
        rowvalue2 = re.sub('\s+[0-9]+$','',curparad[useparad[index+5]])
    print('| ' + defart_forms[useparad[index]],
          rowvalue1,
          #curparad[useparad[index]],
          curparadf[useparad[index]],
          defart_forms[useparad[index+5]],
          rowvalue2,
          #curparad[useparad[index+5]],
          str(curparadf[useparad[index+5]])+' |',file=outf,sep="|")

def printadjrow(useparad,curparad,curparadf,index,outf):
    curcasenum = useparad[index][5].upper()
    if(re.search(',',curparad[useparad[index]])):
        rowvalue1 = curparad[useparad[index]]
    else:
        rowvalue1 = re.sub('\s+[0-9]+$','',curparad[useparad[index]])
    if(re.search(',',curparad[useparad[index+10]])):
        rowvalue2 = curparad[useparad[index+10]]
    else:
        rowvalue2 = re.sub('\s+[0-9]+$','',curparad[useparad[index+10]])
    if(re.search(',',curparad[useparad[index+20]])):
        rowvalue3 = curparad[useparad[index+20]]
    else:
        rowvalue3 = re.sub('\s+[0-9]+$','',curparad[useparad[index+20]])
    print('| ' + curcasenum, 
          #curparad[useparad[index]],
          rowvalue1,
          curparadf[useparad[index]],
          #curparad[useparad[index+10]],
          rowvalue2,
          curparadf[useparad[index+10]],
          #curparad[useparad[index+20]],
          rowvalue3,
          str(curparadf[useparad[index+20]])+' |',file=outf,sep="|")

genders = ['m','f','n']
def printadjparad(curmasc,outf):
    if(not curmasc in lemma_pos_forms):
        print(curmasc,0)
        return

    
    curforms = lemma_pos_forms[curmasc]
    curparad = {}
    curparadf = {}
    for foo in nparads:
        for boo in curforms:
            m = re.search('('+foo+')',boo)
            if(m):
                #print(boo,lemma_pos_forms[curmasc],lemma_pos_forms[curmasc][boo])
                if(foo in curparadf):
                    curparadf[foo] = curforms[boo] + curparadf[foo]
                    if(curparad[foo] == 'NA'):
                        curparad[foo] = re.sub('[^:]+:','',boo)   + ' ' + str(curforms[boo])
                    else:   
                        curparad[foo] = curparad[foo] + ', ' + re.sub('[^:]+:','',boo)  + ' ' + str(curforms[boo])
                        print('hasnum',foo,curparad[foo],curparadf[foo])
                else:
                    curparadf[foo] = curforms[boo]
                    curparad[foo] = re.sub('[^:]+:','',boo) + ' ' + str(curforms[boo])
                print(foo,boo,curparadf[foo])
        for foo in nparads:
            if(not foo in curparad):
                curparad[foo] = 'NA'
                curparadf[foo] = 0
                #print('usetag',usetag)
                
    for foo in curparad:
        print('adj',foo,curparad[foo],curparadf[foo])
    
        
    print('\n',file=outf)
    print('##',curmasc,greeklems[curmasc],file=outf)
    
    print('|case | masc | # | fem  | #| neut  | #|',file=outf)
    print('| --- | --- | --- | --- | --- | --- | ---  |',file=outf)
    print('| Sing | |  |  | |  |   |',file=outf)
    
    for i in range(0,10):
        if(i == 5):
            print('| Plural | |  |  | |  |   |',file=outf)
        print(nparads[i],nparads[i+9],nparads[i+18])
        
        printadjrow(nparads,curparad,curparadf,i,outf)



def printnomparad(curgend,curmasc,outf):
    useparad = []
    for foo in nparad1:
        useparad.append(re.sub('Z',curgend,foo))
    
    if(not curmasc in lemma_pos_forms):
        print(curmasc,0)
        return

    curparad = {}
    curparadf = {}
    for foo in useparad:
        for boo in lemma_pos_forms[curmasc]:
            if(re.search(foo,boo)):
                #print(boo,lemma_pos_forms[curmasc],lemma_pos_forms[curmasc][boo])
                curparad[foo] = re.sub('[^:]+:','',boo) + ' ' + str(lemma_pos_forms[curmasc][boo])
                print('foo',foo,curparad[foo])
                curparadf[foo] = lemma_pos_forms[curmasc][boo]
                #print(foo,boo,curparadf[foo])
        for foo in useparad:
            if(not foo in curparad):
                curparad[foo] = 'NA'
                curparadf[foo] = 0
                
        
    
    
    print('| sg | | # | pl | | #|',file=outf)
    print('| --- | --- | --- | --- | --- | --- |',file=outf)
    
    
    printnounrow(useparad,curparad,curparadf,0,outf)
    printnounrow(useparad,curparad,curparadf,1,outf)
    printnounrow(useparad,curparad,curparadf,2,outf)
    printnounrow(useparad,curparad,curparadf,3,outf)
    printnounrow(useparad,curparad,curparadf,4,outf)

    print('\n\n',file=outf)

def nomgenders(curnom,outf):
    gendtable = {}
    print('\n',file=outf)
    print('##',curnom,greeklems[curnom],file=outf)

    for boo in lemma_pos_forms[curnom]:
        if(boo[0] in ['n','p']):
            curgend = boo[6]
            if(curgend in gendtable):
                gendtable[curgend] = gendtable[curgend] + lemma_pos_forms[curnom][boo]
            else:
                gendtable[curgend] = lemma_pos_forms[curnom][boo]
                
    for foo in gendtable:
        printnomparad(foo,curnom,outf)

            

    
curmasc = 'πολέμιος'
printadjparad(curmasc,outf)
    
curmasc = 'πᾶς'
printadjparad(curmasc,outf)
    
curmasc = 'φίλος'
printadjparad(curmasc,outf)

curmasc = 'στάδιον'
nomgenders(curmasc,outf)
curmasc = 'δῶρον'
nomgenders(curmasc,outf)
curmasc = 'γυνή'
nomgenders(curmasc,outf)
curmasc = 'ἀδελφός'
nomgenders(curmasc,outf)
curmasc = 'ποταμός'
nomgenders(curmasc,outf)

curmasc = 'κώμη'
nomgenders(curmasc,outf)

curmasc = 'μάχη'
nomgenders(curmasc,outf)

curmasc = 'σκηνή'
nomgenders(curmasc,outf)

curmasc = 'σπονδή'
nomgenders(curmasc,outf)

curmasc = 'φυγή'
nomgenders(curmasc,outf)

curmasc = 'φυλακή'
nomgenders(curmasc,outf)

curmasc = 'ἀνήρ'
nomgenders(curmasc,outf)

curmasc = 'παῖς'
nomgenders(curmasc,outf)

printadjparad('ἐπιτήδειος',outf)
nomgenders('στάδιον',outf)
outf.close()


s---mn a-s---mn-:πολέμιος 4
s---mg a-s---mg-:πολεμίου 1
hasnum s---mg πολεμίου 1, πολεμιωτάτου 1 2
s---mg a-s---mgs:πολεμιωτάτου 2
s---ma n-s---ma-:πολέμιον 1
hasnum s---ma πολέμιον 1, πολέμιον 1 2
s---ma a-s---ma-:πολέμιον 2
p---mn a-p---mnc:πολεμιώτεροι 1
hasnum p---mn πολεμιώτεροι 1, πολέμιοι 79 80
p---mn a-p---mn-:πολέμιοι 80
p---mg a-p---mg-:πολεμίων 45
p---md a-p---md-:πολεμίοις 26
p---ma a-p---ma-:πολεμίους 54
hasnum p---ma πολεμίους 54, πολεμίους 1 55
p---ma n-p---ma-:πολεμίους 55
s---fn a-s---fn-:πολεμία 1
s---fg a-s---fg-:πολεμίας 4
s---fd a-s---fd-:πολεμίᾳ 5
s---fa a-s---fa-:πολεμίαν 2
p---fn a-p---fn-:πολέμιαι 1
p---fg a-p---fg-:πολεμίων 2
s---nn a-s---nn-:πολέμιον 1
s---ng a-s---ng-:πολεμίου 1
s---na a-s---na-:πολέμιον 2
p---ng a-p---ng-:πολεμίων 1
p---na a-p---na-:πολέμια 1
adj s---mv NA 0
adj s---mn πολέμιος 4 4
adj s---mg πολεμίου 1, πολεμιωτάτου 1 2
adj s---md NA 0
adj s---ma πολέμιον 1, πολέμιον 1 2
adj p---mv NA 0
adj p---mn πολεμιώτεροι 1, πολέμιοι 79 80
adj p---mg 

In [20]:
import re


tbvoc = {}
tottokens = 0

def addlemma(lem):
    if(lem in tbvoc):
        tbvoc[lem] = tbvoc[lem] + 1
    else:
        tbvoc[lem] = 1
        
def gettbvoc(fname):
    limlems = 69000
    i = 0
    global tottokens
    f = open(fname)
    for l in f:
        if(re.search('postag="u',l)):
            continue
        m = re.search('lemma="([^"]+)"',l)
        if(m):
            i = i + 1
            if(i>limlems):
                break
            curlem = re.sub('#[0-9]*','',m[1])
            addlemma(curlem)
            tottokens = tottokens + 1
        m = re.search("lemma='([^']+)'",l)
        if(m):
            i = i + 1
            if(i>limlems):
                break
            curlem = re.sub('#[0-9]*','',m[1])
            addlemma(curlem)
            tottokens = tottokens + 1
    
    f.close()
  
fname = '/Users/gcrane/github/proiel-treebank/greek-nt.xml'
fname = '/Users/gcrane/github/glaux-trees/public/xml/0011-002.xml'
fname = '/Users/gcrane/github/glaux-trees/public/xml/0003-001.xml'
fname = '/Users/gcrane/github/glaux-trees/public/xml/0059-030.xml'
fname = '/Users/gcrane/github/glaux-trees/public/xml/0032-006.xml'
fname = curtb

gettbvoc(fname)
print(len(tbvoc))

3683


In [21]:
import re
from greek_accentuation.characters import *
#exit

quizoutf = open('quizletvoc.tsv','w')
#do not run!
f = open('crosbyshaeffer2.0.xml')
#outf = open('cs2-2.xml','w')
prevpage = 0
prevpara = 0
curlesson = 0
sawvocab = 0
invocab = 0
gksent = 0
vocabitems = 0
totfreq = 0
csvocab = {}
csglosses = {}
for l in f:
    l = re.sub('@([0-9]+).*','<pb n="\g<1>"/>',l)
    l = re.sub('\n+$','',l)
    m = re.search('<pb n="([0-9]+)',l)
    if(m):
        curpage = int(m[1])
        if(not curpage == prevpage + 1):
            print('jumping from',prevpage,curpage)
        prevpage = curpage
    m = re.search('subtype="para"',l)
    if(m):
        invocab = 0
    if(re.search('^[0-9]+\.\s+.+[α-ω]',l)):
        gksent = gksent + 1
        #print(gksent,'gksent',l,end='')
    if(re.search('VOCAB',l)):
        sawvocab = 1
        invocab = 1
    m = re.search('<div type="textpart" subtype="lesson" n="([0-9]+)">',l)
    if(m):
        if( not sawvocab):
            print('missingvoc',curpage,curlesson)
       
        curlesson = m[1]
       
        sawvocab = 0
        l = re.sub('<div type="textpart" subtype="lesson">','<div type="textpart" subtype="lesson" n="'+str(curlesson)+'">',l)
    curlemma = ''
    m = re.search('^\s*<rs type="lemma" n="([^"]+)"',l)
    if(m):
        curlemma = m[1]
    elif(re.search('^\s*<rs type="lemma">([^<]+)',l)):
        m = re.search('^\s*<rs type="lemma">([^<]+)',l)
        curlemma = m[1]
    if(curlemma):
        vocabitems = vocabitems + 1
        if(not re.search('<rs',l)):
            l = re.sub('^([^ :,;]+)','<rs type="lemma">\g<1></rs>',l)
        curlemma = re.sub('-','',m[1])
        curlemma = strip_length(curlemma)
        csglosses[curlemma] = l
        if(not curlemma in csvocab):
            csvocab[curlemma] = []
        csvocab[curlemma].append(curlesson)
            #print(curlemma)
        if( not  curlemma in tbvoc):
            print('\nfailed',curlemma)
            curfreq = 0
        else:
            curfreq = tbvoc[curlemma]
            totfreq = totfreq + curfreq
        print(vocabitems, curlemma,curpage,curlesson,curfreq,totfreq,l,sep='\t')
        quizl = re.sub('<[^>]+>','',l)
        quizl = re.sub(':\s+','\t',quizl)
        print(vocabitems, curlemma,curpage,curlesson,curfreq,totfreq,quizl,sep='\t',file=quizoutf)
    #print(l,end='',file=outf)

quizoutf.close()

missingvoc 1 0
1	ἀδελφός	2	1	20	20	<rs type="lemma">ἀδελφός</rs>, ἀδελφοῦ, ὁ: brother.  PHILADELPHIA.

failed ἔχει
2	ἔχει	2	1	0	20	<rs type="lemma">ἔχει</rs>: he, she, or it has,

failed ἔχουσι
3	ἔχουσι	2	1	0	20	<rs type="lemma">ἔχουσι</rs>: they have.

failed ἦν
4	ἦν	2	1	0	20	<rs type="lemma">ἦν</rs>: he, she, it, or there was.

failed ἦσαν
5	ἦσαν	2	1	0	20	<rs type="lemma">ἦσαν</rs>: they or there were.

failed παύει
6	παύει	2	1	0	20	<rs type="lemma">παύει</rs>: he, she, or it stops (transitive).

failed παύουσι
7	παύουσι	2	1	0	20	<rs type="lemma">παύουσι</rs>: they stop.

failed πέμπει
8	πέμπει	2	1	0	20	<rs type="lemma">πέμπει</rs>: he, she, or it sends (transitive).

failed πέμπουσι
9	πέμπουσι	2	1	0	20	<rs type="lemma">πέμπουσι</rs>: they send.
10	ποταμός	2	1	99	119	<rs type="lemma">ποταμός</rs>, ποταμοῦ, ὁ: river, HIPPOPOTAMUS.
11	στρατηγός	2	1	151	270	<rs type="lemma">στρατηγός</rs>, στρατηγοῦ, ὁ: general. STRATEGY.
12	ἄνθρωπος	5	2	75	345	<rs type="lemma">ἄνθρωπος</rs>, ἀνθρώπου, 

183	τρόπος	66	21	16	23156	<rs type="lemma">τρόπος</rs>: turn, “bent,” manner, character, TROPIC,
184	ὄνομα	66	21	20	23176	<rs type="lemma">ὄνομα</rs>: name. ONOMATOPOETIC. 
185	φυγάς	66	21	8	23184	<rs type="lemma">φυγάς</rs>, φυγάδος, ἡ: exile, fugitive. Cf. φυγή.
186	παρέχω	66	21	46	23230	<rs type="lemma">παρ-έχω</rs>, παρ-ίξω, παρ-έσχον: furnish, cause. 
187	ὡς	66	21	457	23687	<rs type="lemma">ὡς</rs>, conj, adv.: as, as if, when.
188	εἶτα	69	22	11	23698	<rs type="lemma">εἶτα</rs>, adv.: then, next.
189	οὕτως	69	22	105	23803	<rs type="lemma" n="οὕτως">οὕτω</rs> (before consonants), οὕτως (before vowels), adv. : thus, so, as afore- said. Cf. οὗτος.
190	πᾶς	69	22	250	24053	<rs type="lemma">πᾶς</rs>, πᾶσα, πᾶν: sing., every, whole ; pl. all. PANDEMONIUM.
191	πρῶτος	69	22	100	24153	<rs type="lemma">πρῶτος</rs>: first: πρῶτον, adv. at first. PROTOTYPE. 
192	φόβος	69	22	18	24171	<rs type="lemma">φόβος</rs>: fear. HYDROPHOBIA. 
193	ἀδικέω	72	23	38	24209	<rs type="lemma">ἀ-δικέω</rs>, ἀδικήσ

395	μικρός	137	42	15	38516	<rs type="lemma" n="μικρός">μικρότερος</rs> smaller
396	μικρός	137	42	15	38531	<rs type="lemma" n="μικρός">μικρότατος</rs> smallest
397	μείων	137	42	19	38550	<rs type="lemma">μείων</rs> pl, fewer
398	ἐλαχύς	137	42	14	38564	<rs type="lemma" n="ἐλαχύς">ἐλάττων</rs> Serves also as comparative for ὀλίγος little, few. 
399	ἐλαχύς	137	42	14	38578	<rs type="lemma" n="ἐλαχύς">ἐλάχιστος</rs> least
400	πολύς	137	42	426	39004	<rs type="lemma" n="πολύς">πλείων</rs> more
401	πολύς	137	42	426	39430	<rs type="lemma" n="πολύς">πλέων</rs> more
402	πολύς	137	42	426	39856	<rs type="lemma" n="πολύς">πλεῖστος</rs> PLEONASM. </p></div>
403	πρῶτος	137	42	100	39956	<rs type="lemma">πρῶτος</rs> first
404	ὕστερος	137	42	4	39960	<rs type="lemma">ὕστερος</rs> later, latter 
405	ὕστερος	137	42	4	39964	<rs type="lemma" n="ὕστερος">ὕστατος</rs> last
missingvoc 141 42
406	ἐγγύς	141	43	30	39994	<rs type="lemma">ἐγγύς</rs>, near, nearly 
407	ἐγγύς	141	43	30	40024	<rs type="lemma" n="ἐγγύς">ἐγ

604	δείκνυμι	241	71	10	46356	<rs type="lemma">δείκνυμι</rs>, δείξω, ἔδειξα, δέδειχα, δέδειγμαι, ἐδείχθην: show, point out, PARADIGM.
605	ἐπιδείκνυμι	241	71	18	46374	<rs type="lemma">ἐπι-δείκνῡμι</rs>: show off, display. EPIDEICTIC.
606	ζεύγνυμι	241	71	5	46379	<rs type="lemma">ζεύγνῡμι</rs>, ζεύξω, ἔζευξα, ἔζευγμαι, ἐζεύχθην:i yoke, bind, unite.
607	νεκρός	241	71	11	46390	<rs type="lemma">νεκρός</rs>, νεκροῦ, ὁ: corpse. NECROSIS.
608	ὄμνυμι	241	71	12	46402	<rs type="lemma">ὄμνυμι</rs>, ὀμοῦμαι, ὥμοσα. ὀμώμοκα, ὀμώμοσμαι, ὠμόσθην: swear, take oath.
609	ὀφθαλμός	241	71	6	46408	<rs type="lemma">ὀφθαλμός</rs>, ὀφθαλμοῦ, ὁ: eye. Ch. ὄψομαι. OPHTHALMIA,
610	βλέπω	244	72	3	46411	<rs type="lemma">βλέπω</rs>, βλέψω, ἔβλεψα: look, face, point.
611	ζάω	244	72	24	46435	<rs type="lemma">ζάω</rs>, ζήσω: live; τὸ ζῶον: living thing. ZOOLOGY.
612	γυμνός	244	72	4	46439	<rs type="lemma">γυμνός</rs>, γυμνή, γυμνόν: bare, naked, lightly-clad, GYMNAST.  
613	πονέω	244	72	8	46447	<rs type="lemma">πονέω</rs>,

In [22]:

def totvoclist(startn,lnum):
    vblist=[]
    nomlist = []
    for i in range(startn,lnum+1):
        curchap = str(i)
        
        for boo in csvocab:
            if(boo in stoplist):
                continue
            if( curchap in csvocab[boo]):
                if(re.search('ω$',boo)):
                    vblist.append(boo)
                else:
                    nomlist.append(boo)
    return(vblist,nomlist)

#vblist,nomlist = totvoclist(10)
#vblist12,nomlist12 = totvoclist(12)
#vblist13,nomlist13 = totvoclist(13)
#vblist14,nomlist14 = totvoclist(14)
#vblist15,nomlist15 = totvoclist(15)
#vblist16,nomlist16 = totvoclist(16)
#vblist17,nomlist17 = totvoclist(17)
vblist22,nomlist22 = totvoclist(1,22)
#print(vblist)
#searchvoc(nomlist,[])
#searchvoc(['ὅς'],[])
#vblist12_16 = vblist12 + vblist13 + vblist14 + vblist15 + vblist16
#nomlist12_16 = nomlist12 + nomlist13 + nomlist14 + nomlist15 + nomlist16
#searchvoc(vblist12_16,['iia','pia','pna','fia','fna','aia','ana'])
#nomlist17_18 = nomlist17 + nomlist18
#searchvoc(nomlist18,[])
#searchvoc(['πᾶς'],[])
searchvoc(vblist22,['iia','pia','pna','fia','fna','aia','ana',
                    'ppa','apa','fpa','iie','iip','pie','pim','pip'])
#searchvoc(['ἀπό','ἐκ','ἐν','παρά','εἰς','διά','ἐπί','σύν','κατά','πρός'],[])
#searchvoc(verblist+vbslesson09,['iia','pia','pna','fia','fna'])


1 ['μένω']
new ['τί', 'μένομεν', ';'] μένομεν

2 ['πιστεύω']
new ['ἐπίστευον', 'γὰρ', 'αὐτῷ', '·'] ἐπίστευον

3 ['ἔχω']
new ['εἶχε', 'γὰρ', 'χρυσοῦν', '·'] εἶχε

4 ['ἔχω']
new ['καὶ', 'εἶχεν', 'οὕτως', '.'] εἶχεν

5 ['ἥκω']
new ['ἥξω', 'δὲ', 'ταχέως', '.'] ἥξω

6 ['ἔχω']
new ['οὕτω', 'δὲ', 'ἔχει', '·'] ἔχει

7 ['ἔχω']
new ['ἔχονται', 'οἱ', 'ἄνθρωποι', '·'] ἔχονται

8 ['πείθω']
new ['οἱ', 'δὲ', 'ἡδέως', 'ἐπείθοντο', '·'] ἐπείθοντο

9 ['μένω']
new ['ἐνταῦθα', 'ἔμεινεν', 'ἡμέρας', 'ἑπτά', '·'] ἔμεινεν

10 ['μένω']
new ['ἐνταῦθ’', 'ἔμεινεν', 'ἡμέρας', 'τρεῖς', '·'] ἔμεινεν

11 ['μένω']
new ['ἐνταῦθ’', 'ἔμεινεν', 'ἡμέρας', 'πέντε', '·'] ἔμεινεν

12 ['μένω']
new ['ἐνταῦθα', 'ἔμεινεν', 'ἡμέρας', 'τρεῖς', '.'] ἔμεινεν

13 ['μένω']
new ['ἐνταῦθα', 'ἔμεινε', 'τρεῖς', 'ἡμέρας', '.'] ἔμεινε

14 ['μένω']
new ['ἐνταῦθα', 'ἔμειναν', 'ἡμέρας', 'τρεῖς', '·'] ἔμειναν

15 ['κελεύω']
new ['μεταπέμπεσθαι', 'δ’', 'ἐκέλευεν', 'αὐτόν', '·'] ἐκέλευεν

16 ['μένω']
new ['ἐνταῦθα', 'ἔμειναν', 'ἡμέρας', 'τρεῖς', '

new ['παραινοῦντος', 'δὲ', 'Ξενοφῶντος', 'καὶ', 'τῶν', 'μάντεων', 'συμβουλευόντων', 'ἔδοξε', 'καθῆραι', 'τὸ', 'στράτευμα', '.'] συμβουλευόντων

294 ['κελεύω']
new ['καὶ', 'ὁ', 'Ξενοφῶν', 'ἐκέλευσεν', 'εἰπεῖν', 'τὸν', 'πρῶτον', 'λέξαντα', 'ποῦ', 'καὶ', 'ἐπλήγη', '.'] ἐκέλευσεν

295 ['ἄγω']
new ['ὁποία', 'δὲ', 'μείζων', 'ἐδόκει', 'εἶναι', ',', 'σύνδυο', 'λόχους', 'ἦγον', 'οἱ', 'στρατηγοί', '.'] ἦγον

296 ['λείπω']
new ['ἄλλου', 'δὲ', 'λόχου', 'τῶν', 'δέκα', 'στρατηγῶν', 'τοῦ', 'Ἡγησάνδρου', 'ὀκτὼ', 'μόνους', 'ἔλιπον', '·'] ἔλιπον

297 ['ἔχω']
new ['οἱ', 'μὲν', 'γὰρ', 'Ἕλληνες', 'οὔτε', 'τοξότην', 'εἶχον', 'οὔτε', 'ἀκοντιστὴν', 'οὔτε', 'ἱππέα', '·'] εἶχον

298 ['ἔχω']
new ['λιμὴν', 'δ’', 'ὑπ’', 'αὐτῇ', 'τῇ', 'πέτρᾳ', 'τὸ', 'πρὸς', 'ἑσπέραν', 'αἰγιαλὸν', 'ἔχων', '.'] ἔχων

299 ['ἥκω', 'ἄγω']
new ['καὶ', 'πλοῖον', 'ἐξ', 'Ἡρακλείας', 'ἧκεν', 'ἄλφιτα', 'ἄγον', 'καὶ', 'ἱερεῖα', 'καὶ', 'οἶνον', '.'] ἧκεν
new ['καὶ', 'πλοῖον', 'ἐξ', 'Ἡρακλείας', 'ἧκεν', 'ἄλφιτα', 'ἄγον', 'καὶ', 'ἱερεῖα', 'καὶ', 

new ['ἡμεῖς', 'δέ', ',', 'ὦ', 'ἄνδρες', 'Σινωπεῖς', ',', 'ἥκομεν', 'ἀγαπῶντες', 'ὅτι', 'τὰ', 'σώματα', 'διεσωσάμεθα', 'καὶ', 'τὰ', 'ὅπλα', '·'] ἥκομεν

548 ['διαβαίνω']
new ['τρίτον', 'δὲ', 'Ἅλυν', ',', 'οὐ', 'μεῖον', 'δυοῖν', 'σταδίοιν', ',', 'ὃν', 'οὐκ', 'ἂν', 'δύναισθε', 'ἄνευ', 'πλοίων', 'διαβῆναι', '·'] διαβῆναι

549 ['μέλλω']
new ['εἰ', 'μὲν', 'μέλλει', 'πλοῖα', 'ἔσεσθαι', 'ἱκανὰ', 'ὡς', 'ἀριθμῷ', 'ἕνα', 'μὴ', 'καταλείπεσθαι', 'ἐνθάδε', ',', 'ἡμεῖς', 'ἂν', 'πλέοιμεν', '·'] μέλλει

550 ['ἔχω']
new ['γιγνώσκομεν', 'γὰρ', 'ὅτι', 'ὅπου', 'μὲν', 'ἂν', 'κρατῶμεν', ',', 'δυναίμεθ’', 'ἂν', 'καὶ', 'σῴζεσθαι', 'καὶ', 'τὰ', 'ἐπιτήδεια', 'ἔχειν', '·'] ἔχειν

551 ['λαμβάνω']
new ['οὓς', 'γὰρ', 'παρὰ', 'Κύρου', 'ἔλαβε', 'τρισχιλίους', 'δαρεικοὺς', 'ὅτε', 'τὰς', 'δέκα', 'ἡμέρας', 'ἠλήθευσε', 'θυόμενος', 'Κύρῳ', ',', 'διεσεσώκει', '.'] ἔλαβε

552 ['ἔχω']
new ['ὁμοῦ', 'μὲν', 'ὄντες', 'πολλοὶ', 'ὥσπερ', 'νυνὶ', 'δοκεῖτε', 'ἄν', 'μοι', 'καὶ', 'ἔντιμοι', 'εἶναι', 'καὶ', 'ἔχειν', 'τὰ', 'ἐπιτήδεια', '

788 ['διώκω']
new ['ἐπεὶ', 'δὲ', 'πιεζόμενοι', 'οἱ', 'Ἕλληνες', 'ἐπεχείρησαν', 'διώκειν', ',', 'σχολῇ', 'μὲν', 'ἐπὶ', 'τὸ', 'ἄκρον', 'ἀφικνοῦνται', 'ὁπλῖται', 'ὄντες', ',', 'οἱ', 'δὲ', 'πολέμιοι', 'ταχὺ', 'ἀπεπήδων', '.'] διώκειν

789 ['ἔχω', 'μένω']
new ['ἐνταῦθα', 'ἔμειναν', 'ἡμέρας', 'τρεῖς', 'καὶ', 'τῶν', 'τετρωμένων', 'ἕνεκα', 'καὶ', 'ἅμα', 'ἐπιτήδεια', 'πολλὰ', 'εἶχον', ',', 'ἄλευρα', ',', 'οἶνον', ',', 'κριθὰς', 'ἵπποις', 'συμβεβλημένας', 'πολλάς', '.'] εἶχον
new ['ἐνταῦθα', 'ἔμειναν', 'ἡμέρας', 'τρεῖς', 'καὶ', 'τῶν', 'τετρωμένων', 'ἕνεκα', 'καὶ', 'ἅμα', 'ἐπιτήδεια', 'πολλὰ', 'εἶχον', ',', 'ἄλευρα', ',', 'οἶνον', ',', 'κριθὰς', 'ἵπποις', 'συμβεβλημένας', 'πολλάς', '.'] ἔμειναν

790 ['λαμβάνω', 'κελεύω']
new ['ἐπειδὴ', 'δὲ', 'ἑώρα', 'Χειρίσοφος', 'προκατειλημμένην', 'τὴν', 'ἀκρωνυχίαν', ',', 'καλεῖ', 'Ξενοφῶντα', 'ἀπὸ', 'τῆς', 'οὐρᾶς', 'καὶ', 'κελεύει', 'λαβόντα', 'τοὺς', 'πελταστὰς', 'παραγενέσθαι', 'εἰς', 'τὸ', 'πρόσθεν', '.'] λαβόντα
new ['ἐπειδὴ', 'δὲ', 'ἑώρα', 'Χειρίσοφος', 

new ['ὁ', 'δὲ', 'ἀκούων', 'ταῦτα', 'δύο', 'ἱερεῖα', 'λαβὼν', 'ἐθύετο', 'τῷ', 'Διὶ', 'τῷ', 'βασιλεῖ', 'πότερά', 'οἱ', 'λῷον', 'καὶ', 'ἄμεινον', 'εἴη', 'μένειν', 'παρὰ', 'Σεύθῃ', 'ἐφ’', 'οἷς', 'Σεύθης', 'λέγει', 'ἢ', 'ἀπιέναι', 'σὺν', 'τῷ', 'στρατεύματι', '.'] μένειν

1020 ['ἔχω']
new ['καίτοι', 'αὐτόν', 'σε', 'μάρτυρα', 'σὺν', 'θεοῖς', 'εἰδόσι', 'ποιοῦμαι', 'ὅτι', 'οὔτε', 'ἔχω', 'παρὰ', 'σοῦ', 'ἐπὶ', 'τοῖς', 'στρατιώταις', 'οὐδὲν', 'οὔτε', 'ᾔτησα', 'πώποτε', 'εἰς', 'τὸ', 'ἴδιον', 'τὰ', 'ἐκείνων', 'οὔτε', 'ἃ', 'ὑπέσχου', 'μοι', 'ἀπῄτησα', '·'] ἔχω

1021 ['ἔχω']
new ['ἀλλ’', 'οὔτ’', 'ἐν', 'τοῖς', 'στρατιώταις', 'ἔσει', 'δι’', 'ἐμὲ', 'ἀτιμότερος', 'ἄν', 'τε', 'μένῃς', 'παρ’', 'ἐμοὶ', 'χιλίους', 'μόνους', 'ὁπλίτας', 'ἔχων', ',', 'ἐγώ', 'σοι', 'τά', 'τε', 'χωρία', 'ἀποδώσω', 'καὶ', 'τ-', 'ἄλλα', 'ἃ', 'ὑπεσχόμην', '.'] ἔχων

1022 ['κελεύω']
new ['ἐπειδὴ', 'δὲ', 'πάντας', 'παρήλασε', ',', 'στήσας', 'τὸ', 'ἅρμα', 'πρὸ', 'τῆς', 'φάλαγγος', 'μέσης', ',', 'πέμψας', 'Πίγρητα', 'τὸν', 'ἑρμηνέα', 'πα

new ['τῶν', 'γὰρ', 'στρατιωτῶν', 'οἱ', 'πλεῖστοι', 'ἦσαν', 'οὐ', 'σπάνει', 'βίου', 'ἐκπεπλευκότες', 'ἐπὶ', 'ταύτην', 'τὴν', 'μισθοφοράν', ',', 'ἀλλὰ', 'τὴν', 'Κύρου', 'ἀρετὴν', 'ἀκούοντες', ',', 'οἱ', 'μὲν', 'καὶ', 'ἄνδρας', 'ἄγοντες', ',', 'οἱ', 'δὲ', 'καὶ', 'προσανηλωκότες', 'χρήματα', ',', 'καὶ', 'τούτων', 'ἕτεροι', 'ἀποδεδρακότες', 'πατέρας', 'καὶ', 'μητέρας', ',', 'οἱ', 'δὲ', 'καὶ', 'τέκνα', 'καταλιπόντες', 'ὡς', 'χρήματ’', 'αὐτοῖς', 'κτησάμενοι', 'ἥξοντες', 'πάλιν', ',', 'ἀκούοντες', 'καὶ', 'τοὺς', 'ἄλλους', 'τοὺς', 'παρὰ', 'Κύρῳ', 'πολλὰ', 'καὶ', 'ἀγαθὰ', 'πράττειν', '.'] ἄγοντες

1235 ['διώκω', 'βουλεύω', 'ἀθροίζω']
new ['ἐπεὶ', 'δ’', 'ᾔσθοντο', 'οἱ', 'μὲν', 'Ἕλληνες', 'ὅτι', 'βασιλεὺς', 'σὺν', 'τῷ', 'στρατεύματι', 'ἐν', 'τοῖς', 'σκευοφόροις', 'εἴη', ',', 'βασιλεὺς', 'δ’', 'αὖ', 'ἤκουσε', 'Τισσαφέρνους', 'ὅτι', 'οἱ', 'Ἕλληνες', 'νικῷεν', 'τὸ', 'καθ’', 'αὑτοὺς', 'καὶ', 'εἰς', 'τὸ', 'πρόσθεν', 'οἴχονται', 'διώκοντες', ',', 'ἔνθα', 'δὴ', 'βασιλεὺς', 'μὲν', 'ἁθροίζει', 'τε', 'τοὺς'

In [23]:
vblist22

['ἄγω',
 'γράφω',
 'ἐθέλω',
 'λύω',
 'παύω',
 'φεύγω',
 'φυλάσσω',
 'ἀθροίζω',
 'ἁρπάζω',
 'ἄρχω',
 'μέλλω',
 'διώκω',
 'νομίζω',
 'πείθω',
 'ὑποπτεύω',
 'σπένδω',
 'ἔχω',
 'λείπω',
 'ἀπέχω',
 'ἥκω',
 'σπεύδω',
 'βουλεύω',
 'ἐπιβουλεύω',
 'κελεύω',
 'διαβαίνω',
 'συμβουλεύω',
 'ἐξελαύνω',
 'μένω',
 'οὔπω',
 'διαρπάζω',
 'ἐλαύνω',
 'κωλύω',
 'πιστεύω',
 'τρέπω',
 'λαμβάνω',
 'παρέχω']

In [24]:
i = 0
for foo in csvocab:
    if(not foo in tbvoc):
        i = i + 1
        print(i,foo,csvocab[foo])

1 ἔχει ['1']
2 ἔχουσι ['1']
3 ἦν ['1']
4 ἦσαν ['1']
5 παύει ['1']
6 παύουσι ['1']
7 πέμπει ['1']
8 πέμπουσι ['1']
9 φασί(ν) ['17']
10 φησί(ν) ['17']
11 πότε ['29']
12 ῥήτωρ ['38']
13 θεμιστοκλῆς ['39']
14 χείρων ['42']
15 χείριστος ['42']
16 μνῆμα ['48']
17 ἀλλάσσω ['49']
18 εἴθε ['51']
19 πέποιθα ['52']
20 θνήσκω ['52']
21 φύσις ['54']
22 ἦρα ['64']
23 δίς ['65']
24 διαγιγνώσκω ['72']


In [25]:
i = 0
j = 0
seentokens = 0
for foo in sorted(tbvoc,key=tbvoc.get,reverse=True):
    if(foo[0].isupper()):
        continue
    i= i + 1
    if(i>5000):
        break
    if(not foo in csvocab):
        cv = 'NA'
    else:
        cv = csvocab[foo]
        
    if(   cv == 'NA'):
        j = j + 1
        seentokens = seentokens + tbvoc[foo]
        if(j<500):
            print(i,j,seentokens,tbvoc[foo],foo,cv)
print(tottokens,seentokens,seentokens/tottokens,.8*tottokens)

1 1 6522 6522 ὁ NA
66 2 6644 122 ἀπέρχομαι NA
76 3 6749 105 ὥστε NA
107 4 6827 78 οἴομαι NA
135 5 6884 57 ἀνίστημι NA
140 6 6939 55 σταθμός NA
148 7 6990 51 τρεῖς NA
151 8 7040 50 ἀμφί NA
160 9 7086 46 δύο NA
178 10 7126 40 εἴκοσι NA
186 11 7164 38 προσέρχομαι NA
188 12 7202 38 καταλαμβάνω NA
191 13 7239 37 οἴχομαι NA
194 14 7275 36 καταλείπω NA
197 15 7311 36 τοσοῦτος NA
204 16 7346 35 ἐξέρχομαι NA
205 17 7381 35 ἀπαγγέλλω NA
207 18 7415 34 παραγγέλλω NA
212 19 7449 34 παρέρχομαι NA
217 20 7482 33 ἅπας NA
222 21 7513 31 ἄκρον NA
226 22 7544 31 παραδίδωμι NA
230 23 7574 30 καταβαίνω NA
231 24 7604 30 ὑποζύγιον NA
239 25 7633 29 ἀναβαίνω NA
240 26 7662 29 συλλέγω NA
249 27 7690 28 τριάκοντα NA
258 28 7717 27 ἐπιτίθημι NA
260 29 7744 27 μηδείς NA
262 30 7771 27 συνέρχομαι NA
263 31 7797 26 πρέσβυς NA
264 32 7823 26 καθίστημι NA
267 33 7849 26 ὅσπερ NA
274 34 7874 25 τέσσαρες NA
282 35 7898 24 ἐπέρχομαι NA
312 36 7920 22 διαπράσσω NA
313 37 7942 22 χιών NA
318 38 7963 21 σύμμαχος NA
323 3

777 309 10803 7 εὔζωνος NA
778 310 10810 7 παράγω NA
779 311 10817 7 μόλις NA
780 312 10824 7 διαβιβάζω NA
781 313 10831 7 ὑπερβάλλω NA
782 314 10838 7 ἐπίκειμαι NA
783 315 10845 7 τέως NA
784 316 10852 7 τύρσις NA
785 317 10859 7 παραπλέω NA
786 318 10866 7 ὀρχέομαι NA
787 319 10873 7 μισθοφορά NA
788 320 10880 7 σωφρονέω NA
789 321 10887 7 αἰτία NA
790 322 10894 7 δωρέω NA
792 323 10900 6 διατίθημι NA
793 324 10906 6 ἐκβάλλω NA
794 325 10912 6 συμπράσσω NA
797 326 10918 6 ἀκρόπολις NA
798 327 10924 6 παράδεισος NA
799 328 10930 6 θηρίον NA
800 329 10936 6 ἐρυμνός NA
801 330 10942 6 ἐξέτασις NA
802 331 10948 6 χρύσεος NA
803 332 10954 6 θεωρέω NA
804 333 10960 6 προβάλλω NA
805 334 10966 6 ἐκλείπω NA
806 335 10972 6 κατακόπτω NA
807 336 10978 6 ὑπολείπω NA
808 337 10984 6 κατατίθημι NA
809 338 10990 6 ἀλέξω NA
810 339 10996 6 λυπέω NA
811 340 11002 6 καθεύδω NA
812 341 11008 6 στρατηγέω NA
813 342 11014 6 ὑποψία NA
814 343 11020 6 ναύαρχος NA
815 344 11026 6 τετρακόσιοι NA
816 345 110

In [26]:
csvocab

{'ἀδελφός': ['1'],
 'ἔχει': ['1'],
 'ἔχουσι': ['1'],
 'ἦν': ['1'],
 'ἦσαν': ['1'],
 'παύει': ['1'],
 'παύουσι': ['1'],
 'πέμπει': ['1'],
 'πέμπουσι': ['1'],
 'ποταμός': ['1'],
 'στρατηγός': ['1'],
 'ἄνθρωπος': ['2'],
 'ἄξιος': ['2'],
 'ἀπό': ['2'],
 'δίκαιος': ['2'],
 'Ἑλλήσποντος': ['2'],
 'ἐκ': ['2'],
 'μακρός': ['2'],
 'μικρός': ['2', '42', '42'],
 'πόλεμος': ['2'],
 'πολέμιος': ['2'],
 'φίλος': ['2'],
 'ἀγαθός': ['3', '42', '42', '42', '42'],
 'ἄγγελος': ['3'],
 'ἄγω': ['3'],
 'γράφω': ['3'],
 'ἐθέλω': ['3'],
 'ἵππος': ['3'],
 'καί': ['3'],
 'καλός': ['3', '28', '42', '42'],
 'κίνδυνος': ['3'],
 'λίθος': ['3'],
 'λύω': ['3'],
 'παύω': ['3'],
 'δέ': ['4'],
 'ἐν': ['4'],
 'δένδρον': ['4'],
 'παρά': ['4'],
 'δῆλος': ['4'],
 'δῶρον': ['4'],
 'εἰς': ['4'],
 'πεδίον': ['4'],
 'στάδιον': ['4'],
 'γάρ': ['6'],
 'σπονδή': ['6'],
 'κώμη': ['6'],
 'μάχη': ['6'],
 'φεύγω': ['6'],
 'οὐ': ['6'],
 'φυγή': ['6'],
 'φυλακή': ['6'],
 'σκηνή': ['6'],
 'φυλάσσω': ['6'],
 'ἀγορά': ['7'],
 'ἀρχή': ['7']

In [27]:
paradpats = {}
paradpats['ostem_m1'] = 'form="([^"]+)" lemma="([^"]+ός)" postag="(n.....m[^"]+)'
paradpats['h_hs_h'] = 'form="([^"]+)" lemma="([^"]+[^ίέριε][ηή])" postag="(n.....f[^"]+)'
paradpats['h_hs_La'] = 'form="([^"]+)" lemma="([^"]+[ίέρ][ηα])" postag="(n.....f[^"]+)'
paradpats['defart_m'] = 'form="([^"]+)" lemma="(ὁ)" postag="(l.....m[^"]+)'
paradpats['defart_n'] = 'form="([^"]+)" lemma="(ὁ)" postag="(l.....n[^"]+)'
paradpats['defart_f'] = 'form="([^"]+)" lemma="(ὁ)" postag="(l.....f[^"]+)'
paradpats['wstem_3.pia'] = 'form="([^"]+)" lemma="([^"]+[^άέό]ω)" postag="(v3.pia[^"]+)'
paradpats['wstem_pia'] = 'form="([^"]+)" lemma="([^"]+[^άέό]ω)" postag="(v..pia[^"]+)'
paradpats['wstem_iia'] = 'form="([^"]+)" lemma="([^"]+[^άέό]ω)" postag="(v..iia[^"]+)'
paradpats['wstem_pna'] = 'form="([^"]+)" lemma="([^"]+[^άέό]ω)" postag="(v..pia[^"]+)'
paradpats['regfut_fia'] = 'form="([^"]+)" lemma="([^"ῖῦῶ]+ω)" postag="(v..fia[^"]+)'
paradpats['regfut_fna'] = 'form="([^"]+)" lemma="([^"ῖῦῶ]+ω)" postag="(v..fna[^"]+)'
#lesson 10 covers 1st and 2nd aorists both
paradpats['aor_aia'] = 'form="([^"]+[^ηω])" lemma="([^"]+ω)" postag="(v..aia[^"]+)'
paradpats['aor_ana'] = 'form="([^"]+[^ηω])" lemma="([^"]+ω)" postag="(v..ana[^"]+)'

paradpats['ostem_m2'] = 'form="([^"]+)" lemma="([^"]+ος)" postag="(n.....m[^"]+)'
paradpats['ostem_n'] = 'form="([^"]+)" lemma="([^"]+[οό]ν)" postag="(n.....n[^"]+)'

paradpats['adj1_m'] = 'form="([^"]+)" lemma="([^"]+[οό]ς)" postag="(a.....m[^"]+)'
paradpats['adj1_n'] = 'form="([^"]+)" lemma="([^"]+[οό]ς)" postag="(a.....n[^"]+)'
paradpats['adj1_f'] = 'form="([^"]+)" lemma="([^"]+[οό]ς)" postag="(a.....f[^"]+)'

paradpats['hos_m'] = 'form="([^"]+)" lemma="(ὅς)" postag="(p.....m[^"]+)'
paradpats['hos_n'] = 'form="([^"]+)" lemma="(ὅς)" postag="(p.....n[^"]+)'
paradpats['seautou_m'] = 'form="([^"]+)" lemma="(σεαυτοῦ)" postag="(p.....m[^"]+)'
paradpats['emautou_m'] = 'form="([^"]+)" lemma="(ἐμαυτοῦ)" postag="(p.....m[^"]+)'
paradpats['autos_m'] = 'form="([^"]+)" lemma="(αὐτός)" postag="(a.....m[^"]+)'
paradpats['hode_m'] = 'form="([^"]+)" lemma="(ὅδε)" postag="(a.....m[^"]+)'

#recessive
paradpats['w_p_mp'] = 'form="([^"]+)" lemma="([^"]+ω)" postag="(v..[fp]p[mep][^"]+)'
paradpats['w_a_m'] = 'form="([^"]+)" lemma="([^"]+ω)" postag="(v..ap[me][^"]+)'

lemma_class = {}

lesson01 = ['ostem_m1','defart_m','wstem_3.pia','hos_m','autos_m','hode_m','seautou_m','emautou_m','w_p_mp','w_a_m']
lesson02 = ['ostem_m2','adj1_m','w_p_mp','w_a_m']
lesson03 = ['wstem_pia','wstem_pna']
lesson04 = ['ostem_n','adj1_n','defart_n','hos_n']
lesson06 = ['h_hs_h','adj1_f','defart_f']
lesson07 = ['h_hs_La']
lesson08 = ['wstem_iia']
lesson09 = ['regfut_fia','regfut_fna']
lesson10 = ['aor_aia','aor_ana']

curlesson = lesson06

def add_lemma(pclass,lemma):
    if(not pclass in lemma_class):
        lemma_class[pclass] = {}
    curdict = lemma_class[pclass]
    if(lemma in curdict):
        curdict[lemma] = curdict[lemma] + 1
    else:
        curdict[lemma] = 1
    #lemma_class[pclass] = curdict

def addparad(curpclass,l):
    m = re.search(paradpats[curpclass],l)
    if(m):
        curform = m[1]
        curlemma = m[2]
        curpostag = m[3]
        add_lemma(curpclass,curlemma)
    
def getparadpats(fname):
    global paradpats
    
    f = open(fname)
    for l in f:
        l = re.sub('\n','',l)
        
        for foo in curlesson:
            #print('doing',foo)
            addparad(foo,l)
    
    
    f.close()
 
getparadpats(curtb)

k = 0
for foo in curlesson:
    curdict = lemma_class[foo]

    print(foo,len(curdict))

    i = 0
    j = 0
    for foo in sorted(curdict,key=curdict.get,reverse=True):
        i = i + 1
        j = j + curdict[foo]
        k = k + curdict[foo]
        if(not foo in lexicalentries):
            shortdef = 'nodef'
        else:
            shortdef = lexicalentries[foo]
        print(i,j,k,foo,curdict[foo],shortdef)


h_hs_h 141
1 87 87 κώμη 87 country town
2 121 121 σπονδή 34 drink-offering; (pl.) truce
3 154 154 μάχη 33 battle
4 185 185 ἀνάγκη 31 force
5 214 214 γυνή 29 a woman
6 238 238 δίκη 24 (custom
7 260 260 ἀρχή 22 a beginning
8 282 282 πύλη 22 one wing of a pair of double gates
9 301 301 φυλακή 19 a watching
10 319 319 γνώμη 18 a means of knowing
11 333 333 σκηνή 14 a covered place
12 347 347 κεφαλή 14 the head
13 360 360 οἰκουμένη 13 the inhabited world
14 373 373 κραυγή 13 a crying
15 384 384 λόγχη 11 a spear-head
16 394 394 πηγή 10 running waters
17 403 403 ἀρετή 9 goodness
18 412 412 τιμή 9 that which is paid in token of worth
19 421 421 πέλτη 9 a small light shield
20 429 429 ὑπερβολή 8 a throwing beyond
21 437 437 δείλη 8 afternoon
22 445 445 σφενδόνη 8 a sling
23 452 452 κριθή 7 barley-corns
24 459 459 φυγή 7 flight
25 466 466 Κάλπη 7 nodef
26 472 472 ἐπιβουλή 6 a plan against
27 478 478 Θρᾴκη 6 Thrace
28 483 483 μελίνη 5 millet
29 488 488 νίκη 5 victory
30 493 493 ἐπιστολή 5 a messa

In [28]:
import icu

icu_transliterator = icu.Transliterator.createInstance("Any-Latin")
value = "μῆνιν"
translit_value = icu_transliterator.transliterate(value)
print(translit_value)


mē̂nin


In [29]:
i = 0
for foo in sorted(greeksentlens,key=greeksentlens.get):
    i = i + 1
    print(i,foo,len(greeksentlems[foo]),greeksentlems[foo])
    if(i>10):
        break

1 3668 2 ['φημί', '·']
2 158 3 ['οὗτος', 'λέγω', '·']
3 237 3 ['οὔτε', 'ἀποφεύγω', '·']
4 366 3 ['φημί', 'Ὀρόντας', '.']
5 681 3 ['ἀποκρίνω', 'Κλέαρχος', '·']
6 835 3 ['τίς', 'μένω', ';']
7 972 3 ['καί', 'λέγω', '·']
8 1099 3 ['τίς', 'κατάκειμαι', ';']
9 1264 3 ['ἀνατείνω', 'πᾶς', '.']
10 1280 3 ['δοκέω', 'οὗτος', '.']
11 1423 3 ['ἔξεστι', 'ὁράω', '·']


In [31]:
genabsolutes

['    <word id="9004803" form="Κύρου" lemma="Κῦρος" postag="n-s---mg-" head="9004802" relation="SBJ"/>',
 '    <word id="9005645" form="αὐτοῦ" lemma="αὐτός" postag="a-s---mg-" head="9005643" relation="SBJ"/>']

In [47]:
getfeats('9004803')

9004803 


('', '')

In [41]:
formdict['9004803']

'    <word id="9004803" form="Κύρου" lemma="Κῦρος" postag="n-s---mg-" head="9004802" relation="SBJ"/>'

In [75]:
sorted(preplist)

[['διά', 'a-', 'πολλά', 'πολύς', 'a-p---na-', '1.9.22'],
 ['διά', 'a-', 'ἐπιμέλειαν', 'ἐπιμέλεια', 'n-s---fa-', '1.9.27'],
 ['διά', 'x', 'καὶ', 'καί', 'b--------', '1.9.27'],
 ['διά', 'x', 'ἔχειν', 'ἔχω', 'v--pna---', '1.9.27'],
 ['εἰς', 'a-', 'δικαιοσύνην', 'δικαιοσύνη', 'n-s---fa-', '1.9.16'],
 ['εἰς', 'a-', 'καλλωπισμόν', 'καλλωπισμός', 'n-s---ma-', '1.9.23'],
 ['εἰς', 'a-', 'πόλεμον', 'πόλεμος', 'n-s---ma-', '1.9.14'],
 ['εἰς', 'a-', 'πόλεμον', 'πόλεμος', 'n-s---ma-', '1.9.23'],
 ['εἰς', 'a-', 'πόλεμον', 'πόλεμος', 'n-s---ma-', '1.9.5'],
 ['εἰς', 'a-', 'χώρας', 'χώρα', 'n-p---fa-', '1.9.14'],
 ['εἰς', 'x', ',', ',', 'u--------', '1.9.16'],
 ['εἰς', 'x', '.', '.', 'u--------', '1.9.16'],
 ['εἰς', 'x', 'ἢ', 'ἤ', 'b--------', '1.9.23'],
 ['εἰς', 'x', 'ἢ', 'ἤ', 'b--------', '1.9.23'],
 ['κατά', 'a-', 'μῆνα', 'μείς', 'n-s---ma-', '1.9.17'],
 ['μετά', 'a-', 'Κῦρον', 'Κῦρος', 'n-s---ma-', '1.9.1'],
 ['παρά', 'a-', 'σπονδὰς', 'σπονδή', 'n-p---fa-', '1.9.8'],
 ['παρά', 'a-', 'ὁδοὺς', 'ὁδός'

In [8]:
print(len(aobjverbs))
for foo in sorted(aobjverbs,key=aobjverbs.get,reverse=True):
    print(aobjverbs[foo],foo)

694
341 ἔχω
230 ποιέω
183 ὁράω
158 λαμβάνω
112 λέγω
77 δίδωμι
68 ἄγω
63 ἀκούω
57 πέμπω
43 παρέχω
39 οἶδα
33 αἱρέω
32 δύναμαι
29 πάσχω
28 καλέω
24 καταλείπω
24 πράσσω
23 καταλαμβάνω
21 διαβαίνω
21 φέρω
20 γιγνώσκω
20 καθίστημι
20 παραδίδωμι
20 δέχομαι
19 ἀποκτείνω
19 εὑρίσκω
19 πείθω
18 τίθημι
18 ὑπισχνέομαι
17 καίω
16 φυλάσσω
16 διώκω
16 ἐπαινέω
16 παρακαλέω
16 βούλομαι
15 συλλέγω
15 ἀποδίδωμι
15 συγκαλέω
15 ἀδικέω
14 δέω
14 ἀπαγγέλλω
14 νικάω
13 αἰτέω
13 προδίδωμι
13 ἐρωτάω
13 αἰτιάομαι
11 ἀφαιρέω
11 κατακαίνω
11 φοβέω
10 μεταπέμπω
10 διαβάλλω
10 λύω
10 κατέχω
10 παίω
9 ἀποπέμπω
9 συνάγω
9 καθοράω
9 λείπω
9 πορίζω
9 δοκέω
9 ἔρχομαι
9 ἀφίημι
9 ἀπόλλυμι
9 ἀπαιτέω
9 θάπτω
8 αἰσθάνομαι
8 συμπέμπω
8 διαρπάζω
8 σῴζω
8 κτάομαι
7 κατακαίω
7 ἐάω
7 παρασκευάζω
7 ἐννοέω
7 κελεύω
7 ἐξαπατάω
7 δείκνυμι
7 ἀπολαμβάνω
6 ἀποδείκνυμι
6 συλλαμβάνω
6 βουλεύω
6 ἐπιδείκνυμι
6 ἵστημι
6 τιμάω
6 νομίζω
6 κωλύω
6 δείδω
6 ἀναλαμβάνω
6 χράω
6 ὄμνυμι
6 ἐπίσταμαι
6 βλάπτω
6 διαφθείρω
6 διέρχομαι
6 προσλαμβάνω
6 δι